<a href="https://colab.research.google.com/github/kaybrian/-Database-Design-ALU-Group-work/blob/main/MedicalData_LLms.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:

!pip -qqq install pip --progress-bar off
!pip -qqq install langchain-groq==0.1.3 --progress-bar off
!pip -qqq install langchain==0.1.17 --progress-bar off
!pip -qqq install llama-parse==0.1.3 --progress-bar off
!pip -qqq install qdrant-client==1.9.1  --progress-bar off
!pip -qqq install "unstructured[md]"==0.13.6 --progress-bar off
!pip -qqq install fastembed==0.2.7 --progress-bar off
!pip -qqq install flashrank==0.2.4 --progress-bar off

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires protobuf<5,>=3.20, but you have protobuf 5.27.2 which is incompatible.
google-ai-generativelanguage 0.6.4 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.19.5, but you have protobuf 5.27.2 which is incompatible.
google-api-core 2.16.2 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0.dev0,>=3.19.5, but you have protobuf 5.27.2 which is incompatible.
google-cloud-aiplatform 1.59.0 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.19.5, but you have protobuf 5.27.2 which is incompatible.
google-cloud-bigquery-connection 1.12.1 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.19.5, but y

In [4]:
import os
import textwrap
from pathlib import Path

from google.colab import userdata
from IPython.display import Markdown
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import FlashrankRerank
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Qdrant
from langchain_community.document_loaders import UnstructuredMarkdownLoader
from langchain_community.embeddings.fastembed import FastEmbedEmbeddings
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq
from llama_parse import LlamaParse

os.environ["GROQ_API_KEY"] = userdata.get("GROQ_API_KEY")


def print_response(response):
    response_txt = response["result"]
    for chunk in response_txt.split("\n"):
        if not chunk:
            print()
            continue
        print("\n".join(textwrap.wrap(chunk, 100, break_long_words=False)))

### Get all the books you want into a folder we are going to use

- put all the books in the folder called Data/booos



In [5]:
# get al lthe books
!mkdir data

Create custom instructions for the model to be able to work wint the pdfs we are providing to the model, eg, all the books we need to use

In [6]:
instruction = """
The provided documents contain comprehensive medical information about various drugs, their administration, potential allergic reactions, perceptions on how to take the drugs, and other related medical data.
Your task is to:
1. Extract detailed information about each drug, including its usage, dosage, side effects, contraindications, and interactions.
2. Summarize information about potential allergic reactions associated with different drugs.
3. Provide guidelines on the proper administration of each drug.
4. Extract any additional medical insights, including perceptions and recommendations on how to take the drugs.
5. Ensure all information is accurate, up-to-date, and well-organized.

In addition, engage with the user in a conversational and empathetic manner:
- Start by asking how the user is doing and how they are feeling today.
- Inquire about their age and any specific concerns they might have.
- Offer reassurance and express understanding and empathy towards their situation.
- Provide information and recommendations as if you were a personal doctor, ensuring the user feels heard and cared for.
- Maintain a friendly, approachable, and supportive tone throughout the conversation.

Load and process all the provided PDFs to extract this information and engage with the user in a human-like, conversational manner.
"""


# List of PDF files to be loaded
pdf_files = [
  #  '/content/NLEM.pdf',
  #  '/content/data.pdf',
  #  '/content/data1.pdf',
  #  '/content/data2.pdf',
  #  '/content/data3.pdf',
  #  '/content/data4.pdf',
  #  '/content/data5.pdf',
  #  '/content/data6.pdf',

  # more data on medicine
  '/content/book.pdf',
  '/content/book1.pdf',
  '/content/book2.pdf',
  '/content/book3.pdf',
  '/content/book4.pdf',
  '/content/book5.pdf',
  '/content/book6.pdf',
  '/content/book7.pdf',
  '/content/book8.pdf',
  '/content/book9.pdf',
]


# Initialize the parser with the provided instruction
parser = LlamaParse(
    api_key=userdata.get("LLAMA_PARSE"),
    result_type="markdown",
    parsing_instruction=instruction,
    max_timeout=5000,
)


# Function to load and parse all PDF files
async def load_all_pdfs(pdf_files):
    llama_parse_documents = []
    for pdf_file in pdf_files:
        parsed_data = await parser.aload_data(pdf_file)
        llama_parse_documents.append(parsed_data)
    return llama_parse_documents

# Call the function to load and parse all PDFs
llama_parse_documents = await load_all_pdfs(pdf_files)


Started parsing the file under job_id cac11eca-4d2d-42d8-b1a9-24498161a3d8
Started parsing the file under job_id cac11eca-d6b9-44d9-a484-8079b7eac8d1
Started parsing the file under job_id cac11eca-e090-4702-a6b0-dc651022c7c6
Started parsing the file under job_id cac11eca-f887-4fa3-90f0-3c8fd25540e9
Started parsing the file under job_id cac11eca-654a-448a-85e2-589ce1f01cd9
Started parsing the file under job_id cac11eca-c8cd-4a66-a182-11905dbf88f2
Started parsing the file under job_id cac11eca-2ed1-48ef-9ed7-63835ef2946e
Started parsing the file under job_id cac11eca-5548-4764-ac1f-fb1d07d10073


In [7]:
parsed_doc = llama_parse_documents[0]

In [8]:
print(parsed_doc)

[Document(id_='32fb3103-391a-480f-8efa-2164ccda9151', embedding=None, metadata={'file_path': '/content/NLEM.pdf'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text='National List of Essential Medicines of India 2011\nPage 1 of 123\n---\n# TABLE OF CONTENTS\n\nPREAMBLE ...................................................................................................................................... 4\n\nEXECUTIVE SUMMARY ................................................................................................................... 6\n\nSalient features of NLEM 2011: ............................................................................................................ …..11\n\nThe meetings/deliberations of core committee/ National consultation meetings held for\npreparing the National List of Essential Medicines 2011 ............................................................. 12\n\nPROCESS ADOPTED FOR REVISION OF NLEM .................

In [9]:
Markdown(parsed_doc[0].text[:-1])

National List of Essential Medicines of India 2011
Page 1 of 123
---
# TABLE OF CONTENTS

PREAMBLE ...................................................................................................................................... 4

EXECUTIVE SUMMARY ................................................................................................................... 6

Salient features of NLEM 2011: ............................................................................................................ …..11

The meetings/deliberations of core committee/ National consultation meetings held for
preparing the National List of Essential Medicines 2011 ............................................................. 12

PROCESS ADOPTED FOR REVISION OF NLEM .............................................................................. 13

POTENTIAL USES OF NLEM ........................................................................................................... 15

EXPERTS WHO PARTICIPATED IN PREPARING NLEM- 2011.......................................................... 16

MEDICINES ADDED IN THE NLEM 2011 ....................................................................................... 27

MEDICINES DELETED FROM NLEM 2003 ...................................................................................... 29

NATIONAL LIST OF ESSENTIAL MEDICINES 2011 .......................................................................... 32

|Section: 1 – Anesthesia|32|
|---|---|
|Section: 2 - Analgesics , Antipyretics, Nonsteroidal Anti-inflammatory Medicines, Medicines used to treat Gout and Disease Modifying Agents used in Rheumatoid Disorders|34|
|Section: 3 – Antiallergics and Medicines used in Anaphylaxis|35|
|Section: 4 - Antidotes and Other Substances used in Poisonings|36|
|Section: 5 – Anticonvulsants/ Antiepileptics|37|
|Section: 6 – Anti-infective Medicines|38|
|Section: 7 –Antimigraine medicines|45|
|Section: 8 –Antineoplastic, immunosuppressives and medicines used in palliative care|46|
|Section: 9 –Antiparkinsonism medicines|49|
|Section: 10 –Medicines affecting the blood|50|
|Section: 11 –Blood products and Plasma substitutes|51|
|Section: 12 –Cardiovascular medicines|52|
|Section: 13 –Dermatological medicines (Topical)|55|
|Section: 14 –Diagnostic agents|57|
|Section: 15 –Disinfectants and antiseptics|58|
|Section: 16 –Diuretics|59|

Page 2 of 123
---
# Section: 17 – Gastrointestinal medicines

Page 3 of 123

# Section: 18 – Hormones, other endocrine medicines and contraceptives

# Section: 19 Immunologicals

# Section: 20 – Muscle Relaxants (Peripherally acting) and Cholinesterase Inhibitors

# Section: 21 – Ophthalmological Preparations

# Section: 22 – Oxytocics and Antioxytocics

# Section: 23 – Peritoneal Dialysis Solution

# Section: 24 – Psychotherapeutic Medicines

# Section: 25 – Medicines acting on the respiratory tract

# Section: 26 – Solutions correcting water, electrolyte and acid-base disturbances

# Section: 27 – Vitamins and Minerals

# Alphabetical List of Medicines – Therapeutic area wise

# Alphabetical List of Medicines in NLEM 2011

# List of Medicines in NLEM 2011 – Category Wise
---
PREAMBLE

Essential medicines are those that satisfy the priority healthcare needs of the majority of the population. The essential medicines list needs to be country-specific addressing the disease burden of the nation and the commonly used medicines at primary, secondary, and tertiary healthcare levels. The medicines in the National List of Essential Medicines (NLEM) should be available at affordable costs and with assured quality. The medicines used in the various national health programs, emerging and reemerging infections should be addressed in the list. The Government of India, Ministry of Health & Family Welfare (MOHFW) is mandated to ensure the quality healthcare system by assuring availability of safe and efficacious medicines for its population.

The primary purpose of NLEM is to promote rational use of medicines considering the three important aspects i.e. cost, safety, and efficacy. Furthermore, it promotes prescription by generic names. Healthcare delivery institutions, health insurance bodies, standards setting institutions for medicines, medicine price control bodies, health economists, and other healthcare stakeholders will be immensely benefitted in framing their policies.

Page 4 of 123
---
The NLEM 2011 has been prepared after several rounds of wide consultations with experts of different disciplines from different parts of the country and from various organizations.

The NLEM is a dynamic document and feedback from all stakeholders is welcome which will help in its revision at regular basis.

The core committee of NLEM 2011 would like to place on record its appreciation for the subject experts who made a significant contribution in revision and updating the list of essential medicines of India.

Page 5 of 123
---
# EXECUTIVE SUMMARY

The National list of essential medicines is one of the key instruments in balanced healthcare delivery system of a country which inter alia includes accessible, affordable quality medicine at all the primary, secondary, tertiary levels of healthcare. Realizing this GOI, MOHFW decided to have its own essential medicines list. The first National List of Essential Medicines of India was prepared and released in 1996. This list was subsequently revised in 2003.

To address the issues of changing disease prevalence, treatment modalities, introduction of newer medicines and identification of unacceptable risk-benefit profile as well as therapeutic profile of some medicines, the GOI, MOHFW considered the need for updating the NLEM.

Revision of NLEM was also based on the two important national reference documents i.e., Indian Pharmacopeia 2010 and National Formulary of India, 4th Edition, 2010. While the former deals with the standards of identity, purity and strength of medicines the later provides the information on rational use of medicines particularly for healthcare professionals.

Page 6 of 123
---
Prof. Y. K. Gupta, Head, Department of Pharmacology, All India Institute of Medical Sciences, New Delhi
Prof. S.K. Sharma, Head, Department of Medicine, All India Institute of Medical Sciences, New Delhi
Dr. T.S. Sidhu, Medical Superintendent, Dr. Ram Manohar Lohia Hospital, New Delhi
Dr. A. K. Harit (Chief Medical Officer) representing DDG (M), Dte. GHS, Ministry of Healp & Family Welfare, New Delhi
Dr. D. Kanungo, Addl. DG (Stores) Dte. GHS, Ministry of Healp & Family Welfare, New Delhi
Dr. Chander Shekhar, Scientist F, Indian Council of Medical Research, New Delhi
Dr. Sukarma S. Tanwar, National AIDS Control Organization, New Delhi
Mr. V. K. Tyagi, Deputy Industrial Advisor, Department of Pharmaceuticals, Ministry of Chemicals and Fertilizers, New Delhi
---
# Dr. G. N. Singh, Secretary-cum-Scientific Director, Indian Pharmacopoeia Commission, Ghaziabad

# Dr. Surinder Singh, Drugs Controller General (India), Dte.GHS, Ministry of Health & Family Welfare, New Delhi

The first meeting of the core committee of NLEM was held at CDSCO on July 22, 2010. The meeting was initiated by DCGI, Dr. Surinder Singh, Dr. Y. K. Gupta, Professor & Head, Department of Pharmacology, AIIMS, New Delhi was unanimously nominated as chairman of the Committee.

During this meeting the core committee was informed that the CDSCO had already taken an initiative in this direction and organized a workshop entitled “Expert Group Meeting on Revision and Updating of the National List of Essential Medicines” on September 16, 2009 in collaboration with Department of Pharmacology, AIIMS and WHO India Office. The workshop was attended by representatives from the following: CDSCO, Ministry of Health; WHO; All India Institute of Medical Sciences, New Delhi; Indian Pharmacopoeia Commission; Indian Council of Medical Research and experts of different disciplines from leading medical and pharmacy institutions of the country.

The issue of updating NLEM was also deliberated in a dedicated session during the Annual Conference of Indian Pharmacological Society, December 2009 at Page 8 of 123.
---
Kolkata and the session was chaired by Prof. Y K Gupta, AIIMS and Dr. K Weerasuriya, WHO-SEARO. The core committee took note of the proceedings of these two meetings while revising the NLEM.

In this core committee meeting it was deliberated that although WHO has prepared an updated list of “Essential Medicines”, it cannot be adopted as such. The NLEM of India should be country specific considering the disease prevalence, cost effectiveness of Medicines etc in the country.

In the meeting the criteria for inclusion/ deletion of medicines were developed and a road map for the revision and updating of NLEM, 2003 was drawn.

Sh. V. K. Tyagi from Department of Pharmaceuticals (DoP), Ministry of Chemicals & Fertilizers emphasized the importance of NLEM in view of the directions of the Hon’ble Supreme Court of India. He also mentioned that DoP may rely on the NLEM prepared by MOH &FW for price fixation.

During the meeting it was felt that opinion/views may be taken from across the country by organizing brainstorming regional workshops. However, considering the logistics and time constraints it was decided that a National consultation meet should be organized in Delhi inviting experts from various specialties and from different parts of the country.

Page 9 of 123
---
Accordingly “National Consultation Meet for Revision of National List of Essential Medicines” was organized by Department of Pharmacology, AIIMS & Central Drugs Standard Control Organization (CDSCO), Ministry of Health and Family Welfare on December 3-4, 2010. The meet was supported by WHO India office. Experts from different disciplines from medical and pharmaceutical institutes, hospitals from across the country and concerned government agencies participated. After initial briefing, four groups were formed according to therapeutic areas. The groups were asked to specifically give the reasons/evidence which guided their decision regarding addition/deletion/alteration. Subsequently the recommendations of the individual groups were discussed in the open house. Thereafter the draft recommendations of the Workshop were prepared with general consensus. The recommendations of the Workshop were further deliberated upon by the Expert Core Committee on 4th Jan 2011 and 31st Jan 2011 at CDSCO. The revised NLEM 2011 has thus been prepared.

The Expert Core Committee recommended that all the medicines which are being provided under various National Health Programmes are considered as essential medicines.

Page 10 of 123
---
Salient features of NLEM 2011:

The medicines have been categorized according to therapeutic area. Therefore it is possible that a medicine with more than one indication appears in more than one category.

The issue of mentioning the strength of the medicine dose was deliberated. The committee took the considered view that the strength should be mentioned in the NLEM.

For essentiality of requirement the medicines have been categorized as follows:

P, S and T denote essentiality at Primary, Secondary and Tertiary levels respectively while P, S, T (U in NLEM 2003) indicates essentiality at all pe levels.

A total of 348 medicines (excluding repetitions) are present in NLEM 2011. In the NLEM 2011, 181 medicines fall under the category of P, S and T, 106 medicines fall under the category of S, T while 61 medicines are categorized as T only. In comparison to NLEM 2003, number of medicines deleted is 47 and 43 medicines have been added.

Page 11 of 123
---
# The meetings/deliberations of core committee/National consultation meetings held for preparing the National List of Essential Medicines 2011

# Core Committee Meetings

1. The first meeting was held at CDSCO, FDA Bhawan on July 22, 2010
2. The second meeting was held at CDSCO, FDA Bhawan on January 4, 2011
3. The pird meeting was held at CDSCO, FDA Bhawan on 31st Jan, 2011

# National Consultation Meetings

1. The first consultation meet was held on September 16, 2009 at Department of Pharmacology, AIIMS, New Delhi
2. The second National consultation meet was held on December 3-4, 2010 at Department of Pharmacology, AIIMS, New Delhi

# Other Deliberations:

1. A dedicated session on National Essential Medicine List was organized during 42nd Annual conference of Indian Pharmacological Society, on 11p December, 2009 at Swabhumi, Kolkata
---
# PROCESS ADOPTED FOR REVISION OF NLEM

National List of Essential Medicines 2003

(Base document)

Consultation meetings with Experts

Deliberation on Evidence based criteria for addition and deletion of medicines from the NLEM

Therapeutic area wise group discussion

(Group composition: Clinicians, Pharmacologists, Pharmacists, Scientists and Regulators)

REVISION PROCESS contd...

Page 13 of 123
---
Presentation by groups in open house discussion

Resource Support:

- Scientists, Senior Residents, Junior Residents and PhD Scholar of Dept of Pharmacology, AIIMS, New Delhi

Deliberations/ discussion and reasoning for additions/deletions/modifications

Draft recommendations for NLEM

Consideration and adoption of NLEM by the Core Committee

Resource Materials:

- WHO Model List of Essential Medicines 2010
- WHO model formulary
- National Formulary of India 2010 (Pre print Version)
- Textbooks of Pharmacology, Internal Medicine
- Drug compendia
- Indian Pharmacopoeia
- Internet facility

Page 14 of 123
---
The concept of essential medicines, first introduced by WHO in 1977, has now been adopted by many countries, non-governmental organizations and international non-profit supply agencies. The list is considered to include the most cost-effective medicines for a particular indication. It is developed in concordance with the standard treatment guidelines keeping in mind the healthcare needs of the majority of the population. Careful selection of a limited range of essential medicines results in a higher quality of care, better management of medicines and more cost-effective use of health resources.

The list of essential medicines guides the hospital drug policies, procurement and supply of medicines in the public sector, medicine cost reimbursement, and medicine donations. It helps in monitoring the pricing of medicines. The list serves as a reference document for correct dosage form and strength for prescribing. Preference is given to single drug formulations as opposed to fixed-dose combinations where appropriate. Hence use of NLEM is expected to improve prescribing practices as well as the health outcomes. The appropriate use of medicines selected in the NLEM promotes rational use of medicines. Such rational use of medicines, especially antimicrobial drugs, reduces the development of drug resistance. The list also serves as a reference for assessing the healthcare access of the populace. Lastly, NEML serves as a tool for public education and training of healthcare providers.
---
|S.No|NAME|AFFILIATION|Specialization/Department|Meeting held on September 16, 2009|Meeting held on December 3-4, 2010|
|---|---|---|---|---|---|
|1.|A B RAMTEKE|Joint Drug Controller (India), CDSCO, FDA Bhawan, Kotla Road, New Delhi|Regulatory Body|✓|✓|
|2.|A K PRADHAN|Assistant Drug Controller, CDSCO, FDA Bhawan, Kotla Road, New Delhi|Regulatory Body|✓|✓|
|3.|ANANT MOHAN|Associate Professor, Dept. of Medicine, AIIMS, New Delhi|Medicine|✓|✓|
|4.|ARTI KAPIL|Professor, Dept of Microbiology, AIIMS, New Delhi|Microbiology|✓|✓|
|5.|ARUN K AGARWAL|Dean and Director Professor, Department of ENT, Maulana Azad Medical College, New Delhi|ENT|--|✓|
|6.|ASEEM SAHU|Technical Officer, CDSCO, FDA Bhawan, Kotla Road, New Delhi|Regulatory Body|✓|✓|
|7.|BALRAM BHARGAVA|Professor, Dept. of Cardiology, AIIMS, New Delhi|Cardiology|✓|---|
---
|S.No|NAME|AFFILIATION|Specialization/Department|held on September 16, 2009|held on December 3-4, 2010|
|---|---|---|---|---|---|
|8.|BIKASH MEDHI|Department of Pharmacology, PGIMER, Chandigarh|Pharmacology|✓|✓|
|9.|BISWA MOHAN PADHY|Department of Pharmacology HIMSR, Jamia Hamdard|Pharmacology|√|✓|
|10.|C D TRIPATHI|Dept. of Pharmacology, Vardhman Medical College and Hospital, New Delhi|Pharmacology|✓|--|
|11.|D K SHARMA|AIIMS, New Delhi|Medicine|✓|✓|
|12.|D. KANUNGO|DGHS, MOHFW, Nirman Bhawan, New Delhi|MOHFW|--|✓|
|13.|DEEPIKA DEKA|Dept. of Obstetrics & Gynaecology, AIIMS, New Delhi|Gynaecology|✓|--|
|14.|PARTHASARATHI|JSS College of Pharmacy and JSS Medical College Mysore|Dept. of Clinical Pharmacy Pharmacy|---|✓|
|15.|GAURAV KUMAR|CDSCO, FDA Bhawan, Kotla Road, New Delhi|Regulatory Body|✓|✓|
---
|S.No|NAME|AFFILIATION|Specialization/Department|Meeting held on September 16, 2009|Meeting held on December 3-4, 2010|
|---|---|---|---|---|---|
|16.|GN SINGH|Indian Pharmacopeia Commission, Govt of India, Ghaziabad|Pharmacy|--|✓|
|17.|JAGRITI BHATIA|Dept. of Pharmacology, AIIMS, New Delhi|Pharmacology|✓|√|
|18.|JAI PRAKASH|Indian Pharmacopoea Commission, Govt of India, Ghaziabad|Pharmacy|✓|✓|
|19.|JANARDAN PANDEY|Department of AYUSH, Ministry of health and Family Welfare, Govt of India|MOHFW|---|✓|
|20.|JATINDER KATYAL|Dept. of Pharmacology, AIIMS, New Delhi|Pharmacology|✓|✓|
|21.|JITENDRA|Department of Hospital Administration, AIIMS, New Delhi|Hospital Administration|--|✓|
|22.|K D BARMAN|Department of Dermatology & STD, Maulana Azad Medical College, New Delhi|Dermatology and STD|--|✓|
|23.|K H REETA|Dept. of Pharmacology, AIIMS, New Delhi|Pharmacology|✓|✓|

Page 18 of 123
---
|S.No|NAME|AFFILIATION|Specialization/Department|Meeting held on September 16, 2009|Meeting held on December 3-4, 2010|
|---|---|---|---|---|---|
|24.|K K PANT|Dept. of Pharmacology, CSM Medical College and Hospital, Lucknow|Pharmacology|✓|✓|
|25.|K K SHARMA|Dept. of Pharmacology, UCMS, New Delhi|Pharmacology|✓|---|
|26.|K WEERASURIYA|Regional Adviser Essential Drugs and Medicines Policy World Health Organization – SEARO|Clinical Pharmacology|✓|--|
|27.|KAMAL KISHORE|Dept. of Pharmacology, AIIMS, New Delhi|Pharmacology|✓|--|
|28.|M C GUPTA|Dept. of Pharmacology, PGIMS, Rohtak, Haryana|Pharmacology|✓|--|
|29.|MADHUR GUPTA|WHO country office for India|Clinical Pharmacology|--|✓|
|30.|MADHURI BEHARI|Dept. of Neurology, AIIMS, New Delhi|Neurology|✓|✓|
|31.|MANJARI TRIPATHI|Dept. of Neurology, AIIMS, New Delhi|Neurology|✓|--|

Page 19 of 123
---
|S.No|NAME|AFFILIATION|Specialization/Department|held on September 16, 2009|held on December 3-4, 2010|
|---|---|---|---|---|---|
|32.|MK DAGA|Department of Medicine, Maulana Azad Medical College, New Delhi|Medicine|---|✓|
|33.|MPS CHAWLA|Department of Medicine, RML Hospital, New Delhi|Medicine|---|✓|
|34.|MUKUL MATHUR|Dept. of Pharmacology, SMS Medical college, Jaipur|Pharmacology|---|✓|
|35.|N R BISWAS|Dept. of Pharmacology, AIIMS, New Delhi|Clinical Pharmacology|✓|--|
|36.|NARESH KHANNA|Dept. of Pharmacology, UCMS, New Delhi|Pharmacology|✓|✓|
|37.|NASEEM SHAH|Chief, Professor and Head, Centre for Dental Education and Research, AIIMS, New Delhi|Dental Medicine|✓|---|
|38.|AJOY ROYCHAUDHURY|Associate Professor, Centre for Dental Education and Research, AIIMS, New Delhi|Dental Medicine|---|✓|
|39.|NEENA MALHOTRA|Department of Obstretics and Gynaecology, AIIMS, New Delhi|Obstetrics and gynecology|---|✓|
|40.|NEENA VALECHA|Dy. Director, National Institute of Malaria Research, Dwarka, New Delhi|Pharmacology and Malaria expertise|✓|-√--|
---
|S.No|NAME|AFFILIATION|Specialization/Department|held on September 16, 2009|held on December 3-4, 2010|
|---|---|---|---|---|---|
|41.|O P AGARWAL|Indian Council of Medical Research, New Delhi|Pharmacology|✓|---|
|42.|P K MEDIRATTA|Dept. of Pharmacology, UCMS, New Delhi|Pharmacology|✓|✓|
|43.|P USHA RANI|Nizams Institute of Medical Sciences, Hyderabad|Clinical Pharmacology|---|✓|
|44.|PRAMIL TIWARI|Dept. of Pharmacy Practice, NIPER, SAS Nagar, Mohali, Punjab|Pharmacy|✓|---|
|45.|PRAMOD GARG|Department of Gastroenterology and Human Nutrition, AIIMS, New Delhi|Physician|---|✓|
|46.|PRAVEEN AGGARWAL|Dept. of Emergency Medicine, AIIMS, New Delhi|Medicine|✓|✓|
|47.|R C DEKA|AIIMS, New Delhi.|Health Administrator and ENT|✓|√---|
|48.|R K BATRA|Dept. of Anaesthesia and Intensive Care, AIIMS, New Delhi|Anaesthesia|✓|✓|

Page 21 of 123
---
|S.No|NAME|AFFILIATION|Specialization/Department|Meeting held on September 16, 2009|Meeting held on December 3-4, 2010|
|---|---|---|---|---|---|
|49.|RAJENDRA PRASAD|Dept of Pulmonary Medicine, CSM Medical College and Hospital, Lucknow|Pulmonary medicine|---|✓|
|50.|RAJNI KAUL|Indian Council of Medical Research, New Delhi|Basic Scientist|✓|---|
|51.|RAKESH CHADDHA|Department of Psychiatry, AIIMS|Psychiatry|---|✓|
|52.|RAKESH YADAV|Dept of Cardiology, AIIMS, New Delhi|Cardiology|✓|✓|
|53.|RESHU AGARWAL|Programme Officer (CST), NACO, New Delhi|Medicine|---|✓|
|54.|ROHIT SAXENA|Associtate Professor, Dr. R P Centre for Opthalmic Sciences, AIIMS|Ophthalmology|---|✓|
|55.|S K TRIPATHI|Professor and Head, Dept. of Clinical Pharmacology, School of Tropical Medicine, Kolkata, WB|Clinical Pharmacology|✓|✓|
|56.|SAMEER BAKSHI|Associate Professor, Medical Oncology, IRCH, AIIMS, New Delhi|Medical Oncology|✓|✓|

Page 22 of 123
---
|S.No|NAME|AFFILIATION|Specialization/Department|Meeting held on September 16, 2009|Meeting held on December 3-4, 2010|
|---|---|---|---|---|---|
|57.|SANDEEP KAUSHAL|Department of Pharmacology, Dayanand Medical College, Ludhiana|Pharmacology|✓|---|
|58.|SANGEETA SHARMA|Dept of Neuropsycho Pharmacology, IHBAS, Delhi|Clinical Pharmacology|✓|✓|
|59.|SANJAY KR. RAI|Centre for Community Medicine AIIMS, New Delhi.|Community Medicine|---|✓|
|60.|SEEMA DUBEY|Institute of Liver and Biliary Sciences New Delhi|Pharmacy|---|✓|
|61.|SHAKTI GUPTA|R.P. Centre for Ophthalmic Sciences & Head, Dept. of Hosp. Administration, AIIMS, New Delhi|Hospital Administration|✓|✓|
|62.|SHARDA PESHIN|Dept. of Pharmacology, AIIMS, New Delhi|Toxicology|✓|✓|
|63.|SHRIDHAR DWIVEDI|Department of Medicine UCMS, New Delhi|Medicine|---|✓|

Page 23 of 123
---
|S.No|NAME|AFFILIATION|Specialization/Department|Meeting held on September 16, 2009|Meeting held on December 3-4, 2010|
|---|---|---|---|---|---|
|64.|SHYAM SUNDER SHARMA|Department of Pharmacology, NIPER, Mohali|Pharmacology|---|✓|
|65.|SK MAULIK|Department of Pharmacology AIIMS, New Delhi|Pharmacology|---|✓|
|66.|SK PRADHAN|DGHS, MOHFW New Delhi|Community Medicine|---|✓|
|67.|SUNIL KULSHESTRA|CDSCO, FDA Bhawan, Kotla Road, New Delhi|Pharmacy Regulatory Body|✓|✓|
|68.|SUNIL NANDRAJ|National Professional Officer (Health Systems Development), WHO, New Delhi|World Health Organization|✓|---|
|69.|SURENDER SINGH|Dept. of Pharmacology, AIIMS, New Delhi|Pharmacy and Pharmacology|✓|✓|
|70.|T.S. SIDHU|Medical Superintendent, RML Hospital, New Delhi|ENT surgeon and Hospital Administration|---|✓|
|71.|TUSHAR AGGARWAL|Assistant Professor, Department of Ophthalmology AIIMS, New Delhi|Ophthalmology|---|✓|
|72.|U S GARG|Department of Hospital Administration, AIIMS, New Delhi|Hospital Administration|---|✓|

Page 24 of 123
---
|S.No|NAME|AFFILIATION|Specialization/Department|Meeting held on September 16, 2009|Meeting held on December 3-4, 2010|
|---|---|---|---|---|---|
|73.|UMA TEKUR|Maulana Azad Medical College, New Delhi|Professor & Head, Department of Pharmacology|---|✓|
|74.|V K PAUL|AIIMS, New Delhi|Professor & Head, Department of Pediatrics|✓|---|
|75.|V. K. TYAGI|Ministry of Chemical & Fertilisers, New Delhi|Deputy Industrial Advisor, Dept. of Pharmaceuticals, Ministry of Chemicals & Fertilizers|---|✓|
|76.|VANDANA JAIN|AIIMS, New Delhi|Assistant Professor, Dept. of Pediatrics|✓|---|
|77.|VIJAY KUMAR|Indian Council of Medical Research, New Delhi|Scientist F, Health administrator|✓|---|
|78.|VINAY GULATI|AIIMS, New Delhi|Assistant Professor, Department of Medicine|✓|---|
|79.|VINOD RAINA|AIIMS, New Delhi|Professor and Head, Medical Oncology, Institute Cancer Rotary Hospital (IRCH)|✓|---|
| |VISHAL KUMAR MALHOTRA|Chacha Nehru Bal Chikitsalaya, New Delhi|Medical Officer, Pediatrics|---|✓|
---
|S.No|NAME|AFFILIATION|Specialization/Department|held on September 16, 2009|held on December 3-4, 2010|
|---|---|---|---|---|---|
|81.|VIVEK ARYA|Department of Medicine, PGIMER, Dr. RML Hospital, New Delhi|Medicine|---|✓|
|82.|Y K GUPTA|Dept of Pharmacology, AIIMS, New Delhi|Pharmacology|✓|✓|
|83.|YASHPAL SHARMA|Department of Cardiology, PGIMER, Chandigarh|Cardiology|---|✓|
|84.|Pooja Gupta|Department of Pharmacology, AIIMS, New Delhi|Clinical Pharmacology|✓|✓|
|85.|Ashish Kumar Kakkar|Department of Pharmacology, AIIMS, New Delhi|Clinical Pharmacology|---|✓|
|86.|Sudhir Chandra Sarangi|Department of Pharmacology, AIIMS, New Delhi|Clinical Pharmacology|✓|✓|
|87.|Saravana Perumal S|Department of Pharmacology, AIIMS, New Delhi|Pharmacy|✓|✓|
---
|S.No.|Name of the medicine|
|---|---|
|1.|25% Dextrose|
|2.|5-Amino salicylic Acid (5-ASA)|
|3.|Allopurinol|
|4.|Amoxicillin+Clavulinic acid|
|5.|Atorvastatin|
|6.|Betamethasone|
|7.|Carboplatin|
|8.|Cefixime|
|9.|Cetrizine|
|10.|Chlorambucil|
|11.|Clindamycin|
|12.|Clopidogrel|
|13.|Dacarbazine|
|14.|Daunorubicin|
|15.|Diazepam|
|16.|EMLA cream|
|17.|Enoxaparin|
|18.|Famotidine|
|19.|Fentanyl|
|20.|Filgrastim|
|21.|Hydroxychloroquine phosphate|
|22.|Ifosfamide|
|23.|Imatinib|
---
|24.|Ipratropium bromide|
|---|---|
|25.|Leflunomide|
|26.|Lorazepam|
|27.|Mefloquine|
|28.|Mesna|
|29.|Misoprostol|
|30.|N-acetylcysteine|
|31.|Olanzapine|
|32.|Oxaliplatin|
|33.|Pantoprazole|
|34.|Permethrin|
|35.|Piperazine|
|36.|Premix Insulin 30:70 injection|
|37.|Propofol|
|38.|Sevoflurane|
|39.|Stavudine+Lamivudine|
|40.|Tramadol|
|41.|Vecuronium|
|42.|Zidovudine+Lamivudine+Nevirapine|
|43.|Zinc Sulfate|
---
|S.No.|Name of the medicine|
|---|---|
|1.|Acenocoumarol|
|2.|Aminophylline|
|3.|Benzoic Acid + Salicylic Acid|
|4.|Benzylpenicillin|
|5.|Bretylium Tosylate|
|6.|Cefuroxime|
|7.|Centchroman|
|8.|Chloramphenicol|
|9.|Chlorthalidone|
|10.|Clarithromycin|
|11.|Clomipramine hydrochloride|
|12.|Clonidine Hydrochloride|
|13.|Doxapram|
|14.|Ethyl Chloride|
|15.|Flucytosine|
|16.|Furazolidone|
|17.|Gamma Benzene Hexachloride|
|18.|Isoprenaline Hydrochloride|
|19.|Isoxsuprine hydrochloride|
|20.|Ketoconazole|
|21.|Loperamide|
|22.|Mebendazole|

Page 29 of 123
---
|S.No.|Name of the medicine|
|---|---|
|23.|Menadione Sodium Sulphite|
|24.|Mexiletine Hydrochloride|
|25.|Nalidixic Acid|
|26.|Niclosamide|
|27.|Nitrazepam|
|28.|Norfloxacin|
|29.|Pancuronium|
|30.|Pentazocine|
|31.|Pethidine Hydrochloride|
|32.|Physostigmine Salicylate|
|33.|Procaine Benzylpenicillin|
|34.|Prochlorperazine|
|35.|Pyrantel Pamoate|
|36.|Quinidine|
|37.|Roxithromycin|
|38.|Silver Nitrate|
|39.|Terazosin|
|40.|Tetracycline|
|41.|Tetracycline Hydrochloride|
|42.|Theophyline compounds|
|43.|Thiacetazone + Isoniazid|
|44.|Tinidazole|
|45.|Trifluoperazine|
---
|S.No.|Name of the medicine|
|---|---|
|46.|Trimethoprim|
|47.|Xylometazoline|

Page 31 of 123
---
|Medicines|Category|Route of Administration/Dosage Form|Strengths|
|---|---|---|---|
|Ether|S, T|Inhalation|--|
|Halothane with vaporizer|S, T|Inhalation| |
|Isoflurane|S, T|Inhalation| |
|Ketamine Hydrochloride|P, S, T|Injection|10 mg / ml, 50 mg / ml|
|Nitrous Oxide|P, S, T|Inhalation| |
|Oxygen|P, S, T|Inhalation| |
|Thiopentone Sodium|S, T|Injection|0.5 g, 1 g powder|
|Sevoflurane|T|Inhalation| |
|Propofol|P, S, T|Injection|1% oil suspension|

|Medicines|Category|Route of Administration/Dosage Form|Strengths|
|---|---|---|---|
|Bupivacaine Hydrochloride|S, T|Injection|0.25%, 0.5%, 0.5% to be mixed with 7.5% glucose solution|
---
# Topical Forms

|Lignocaine Hydrochloride|P, S, T|Injection|1-2%|
|---|---|---|---|
| | |Spinal|5% + 7.5% Glucose|

# Added Medicines

EMLA cream
T
Cream

# 1.3 Preoperative Medication and Sedation for Short Term Procedures

|Medicines|Category|Route of Administration/Dosage Form|Strengths|
|---|---|---|---|
|Atropine Sulphate|P, S, T|Injection|0.6 mg / ml|
| |P, S, T|Tablets|5 mg|
|Diazepam|S, T|Syrup|2mg/5ml|
| | |Suppository|5 mg|
| | | |1 mg / ml|
|Midazolam|P, S, T|Injection|5 mg / ml|
|Morphine Sulphate|S, T|Injection|10 mg / ml|
|Promethazine|P, S, T|Syrup|5 mg / 5 ml|

Page 33 of 123
---
# Section: 2 - Analgesics, Antipyretics, Nonsteroidal Anti-inflammatory Medicines, Medicines used to treat Gout and Disease Modifying Agents used in Rheumatoid Disorders

|Medicines|Category|Route of Administration/Dosage Form|Strengths|
|---|---|---|---|
|Acetyl Salicylic Acid|P, S, T|Tablets|325mg, 350 mg|
|Diclofenac|T|Tablets|50 mg|
|Diclofenac|T|Injection|25 mg/ml|
|Diclofenac|P, S, T|Tablets|200 mg, 400 mg|
|Ibuprofen|P, S, T|Syrup|100mg/5ml|
|Ibuprofen|P, S, T|Injection|150 mg/ml|
|Paracetamol|P, S, T|Syrup|125 mg/5ml|
|Paracetamol|P, S, T|Tablets|500 mg|
|Paracetamol|P, S, T|Suppository|80 mg, 170 mg|

# 2.2 Opioid Analgesics

|Medicines|Category|Route of Administration/Dosage Form|Strengths|
|---|---|---|---|
|Morphine Sulphate|S, T|Injection|10 mg/ml|
|Morphine Sulphate|S, T|Tablets|10 mg|
|Tramadol|S, T|Injection|50 mg/ml|
|Tramadol|S, T|Cap|50 mg, 100 mg|
|Fentanyl|S, T|Injection|50μg/ml 2ml ampoule|
---
# Medicines used to treat Gout

|Allopurinol|S, T|Tablets|100 mg|
|---|---|---|---|
|Colchicine|S, T|Tablets|0.5 mg|

# Disease modifying agents used in Rheumatoid disorders

|Azathioprine|S, T|Tablets|50 mg|
|---|---|---|---|
|Methotrexate|S, T|Tablets|5mg, 7.5mg, 10mg|
|Sulfasalazine|S, T|Tablets|500 mg|

# Added medicines

|Hydroxychloroquine phosphate|S, T|Tablets|200 mg|
|---|---|---|---|
|Leflunomide|S, T|Tablets|10mg ,20 mg tab|

# Antiallergics and Medicines used in Anaphylaxis

|Medicines|Category|Route of Administration/Dosage Form|Strengths|
|---|---|---|---|
|Adrenaline Bitartrate|P, S, T|Injection|1 mg / ml|
|Chlorpheniramine Maleate|P, S, T|Tablets|4 mg|
|Dexchlorpheniramine Maleate|P, S, T|Syrup|0.5 mg / 5 ml|
|Dexamethasone|P, S, T|Tablets|0.5 mg|
|Dexamethasone|P, S, T|Injection|4 mg / ml|
|Hydrocortisone Sodium Succinate|P, S, T|Injection|100 mg|
|Pheniramine Maleate|P, S, T|Injection|22.75 mg / ml|
|Prednisolone|P, S, T|Tablets|5 mg, 10 mg, 20 mg|
|Promethazine|P, S, T|Tablets|10 mg, 25 mg|
|Promethazine|P, S, T|Syrup|5 mg / 5 ml|

# Added Medicines

|Cetrizine|P, S, T|Tablets|10mg|
|---|---|---|---|
|Cetrizine|P, S, T|Syrup|5 mg/ml|
---
|Medicines|Category|Route of Administration/Dosage Form|Strengths|
|---|---|---|---|
|Activated Charcoal|P,S,T|Oral| |
|Atropine Sulphate|P,S,T|Injection|1 mg/ml|
|Specific Antisnake venom|P,S,T|Injection Polyvalent Solution/ Lyophilyzed Polyvalent Serum| |
|Calcium gluconate|P,S,T|Injection|100mg/ml|
|Desferrioxamine mesylate|S, T|Injection|500mg|
|Methylthioninium chloride (Methylene blue)|S, T|Injection|10 mg / ml|
|Penicillamine|S, T|Tablets or Capsules|250 mg|
|Dimercaprol|S, T|Injection in oil|50 mg / ml|
|Flumazenil|T|Injection|0.1 mg / ml|
|Sodium Nitrite|S, T|Injection|30 mg / ml|
|Sodium Thiosulphate|S, T|Injection|250 mg/ ml|
|Naloxone|P,S,T|Injection|0.4mg/ml|
|Pralidoxime Chloride(2-PAM)|P,S,T|Injection|25 mg/ml|
|N-acetylcysteine|P,S,T|Injection|200 mg/ml (5 ml)|
---
|Medicines|Category|Strengths|Dosage Form|
|---|---|---|---|
|Carbamazepine|P, S, T|Tablets|100mg|
| | |Syrup|200mg|
| | | |100 mg/5ml|
|Diazepam|P, S, T|Injection|5 mg / ml|
|Magnesium sulphate|S, T|Injection|500 mg /ml|
|Phenobarbitone|P, S, T|Tablets|30 mg, 60 mg|
| | |Injection|200 mg/ml|
| | |Syrup|20 mg/5ml|
|Phenytoin Sodium|P, S, T|Capsules or Tablets|50 mg|
| | |Syrup|100mg|
| | |Injection|25mg/ml|
| | | |50 mg/ml|
|Sodium Valproate|P, S, T|Tablets|200 mg|
| | |Syrup|500mg|
| | | |200 mg/5ml|
| |T|Injection|100 mg/ml|
|Lorazepam|T|Injection|2mg/ml|
---
# Section: 6 – Anti-infective Medicines

# 6.1 Anthelminthics

|Medicines|Category|Route of Administration/Dosage Form|Strengths|
|---|---|---|---|
|Albendazole|P,S,T|Tablets|400 mg|
| | |Suspension|200 mg/5 ml|
|Piperazine|P,S,T|Tablets|4.5 gm|
| | |Solution|750 mg/5 ml|

# 6.1.2 Antifilarials

|Medicines|Category|Route of Administration/Dosage Form|Strengths|
|---|---|---|---|
|Diethylcarbamazine citrate|P,S,T|Tablets|50 mg|

# 6.1.3 Antischistosomals and Antitrematode Medicines

|Medicines|Category|Route of Administration/Dosage Form|Strengths|
|---|---|---|---|
|Praziquantel|S, T|Tablets|600 mg|

# 6.2 Antibacterials

# 6.2.1 Beta lactam medicines

|Medicines|Category|Route of Administration/Dosage Form|Strengths|
|---|---|---|---|
|Amoxicillin|S,T|Powder for suspension|125 mg / 5 ml|
| | |Capsules|250 mg, 500 mg|
|Ampicillin|P,S,T|Powder for suspension|125 mg / 5 ml|
| | |Injection|500 mg|
---
# Antibacterials

|Medicines|Category|Route of Administration/Dosage Form|Strengths|
|---|---|---|---|
|Amikacin|S, T|Injection|250 mg / 2 ml|
|Azithromycin|S, T|Suspension|100mg/5ml|
| | |Injection|500mg|
|Ciprofloxacin Hydrochloride|P,S,T|Injection|200 mg /100 ml|
| | |Tablets|250 mg, 500 mg|
|Co-Trimoxazole (Trimethoprim + Sulphamethoxazole)|P,S,T|Tablets|80 + 400 mg, 160+800 mg|
| | |Suspension|40 + 200 mg / 5 ml|

# Added Medicines

|Medicines|Category| | |
|---|---|---|---|
|Amoxicillin + Clavulinic acid|T|Tablets|625 mg|
| | |Powder for suspension|228.5mg/5ml|
| | |Injection|600mg, 1.2gm|
|Cefixime|T|Tablet|100, 200mg|

# Antibacterials

|Medicines|Category|Route of Administration/Dosage Form|Strengths|
|---|---|---|---|
|Benzathine Benzylpenicillin|P,S,T|Injection|6 lacs, 12 lacs units|
|Cefotaxime|S, T|Injection|125 mg, 250 mg, 500 mg|
|Ceftazidime|S, T|Injection|250mg, 1g|
|Ceftriaxone|S, T|Injection|250 mg, 1 g|
|Cephalexin|P,S,T|Syrup|125 mg / 5 ml|
| | |Capsules|250 mg, 500 mg|
|Cloxacillin|P,S,T|Capsules|250 mg, 500 mg|
| | |Injection|250 mg|
| | |Liquid|125mg/ 5 ml|
---
# Antimicrobial Medicines

|Medicines|Category|Route of Administration/Dosage Form|Strengths|
|---|---|---|---|
|Doxycycline|P,S,T|Tablets|100 mg|
|Erythromycin Estolate|P,S,T|Tablets|250 mg, 500 mg|
|Gentamicin|P,S,T|Injection|10 mg/ml, 40 mg/ml|
|Metronidazole|P,S,T|Tablet|200 mg, 400 mg|
| | |Injection|500 mg/100ml|
| | |Syrup|100 mg/5ml|
|Nitrofurantoin|P,S,T|Tablets|100 mg|
|Sulphadiazine|S, T|Tablets|500 mg|
|Vancomycin Hydrochloride|T|Injection|500 mg, 1 g|

# Antileprosy Medicines

|Medicines|Category|Route of Administration/Dosage Form|Strengths|
|---|---|---|---|
|Clofazimine|P,S, T|Capsules|50 mg, 100 mg|
|Dapsone|P,S, T|Tablets|50 mg, 100 mg|
|Rifampicin|P,S, T|Capsules or Tablets|150 mg, 300 mg|

# Antituberculosis Medicines

|Medicines|Category|Route of Administration/Dosage Form|Strengths|
|---|---|---|---|
|Ethambutol|P,S,T|Tablets|200 mg, 400 mg, 600 mg, 800 mg|
| | |Tablets|50 mg, 100 mg, 300 mg|
|Isoniazid|P,S,T|Syrup|100 mg/5ml|
| | |Tablets|100 mg, 200 mg|
|Ofloxacin|S, T|Syrup|50 mg/5 ml|
| | |Tablets|500 mg, 750 mg, 1000 mg, 1500 mg|
---
|Medicines|Category|Route of Administration/Dosage Form|Strengths|
|---|---|---|---|
|Amphotericin B|S, T|Injection|50 mg|
|Amphotericin B|S, T|Pessaries|100 mg, 200 mg|
|Clotrimazole|P, S, T|Gel|2%|
|Fluconazole|S, T|Capsules or Tablets|50 mg, 100 mg, 150 mg, 200 mg|
|Griseofulvin|P, S, T|Capsules or Tablets|125 mg, 250 mg|
|Nystatin|P, S, T|Pessaries|100,000 IU|

|Medicines|Category|Route of Administration/Dosage Form|Strengths|
|---|---|---|---|
|Acyclovir|S, T|Tablets|200 mg, 400 mg|
|Acyclovir|S, T|Injection|250 mg, 500 mg|
|Acyclovir|S, T|Suspension|400 mg / 5 ml|

|Medicines|Category|Route of Administration/Dosage Form|Strengths|
|---|---|---|---|
|Didanosine|S, T|Tablets|250 mg, 400 mg|
---
|Medicines|Category|Route of Administration/Dosage Form|Strengths|
|---|---|---|---|
|Lamivudine|S, T|Tablets|150 mg|
|Lamivudine + Nevirapine + Stavudine|S, T|Tablets|150 mg + 200 mg + 30 mg|
|Lamivudine + Zidovudine|S, T|Tablets|150 mg + 300 mg|
|Stavudine|S, T|Capsules|15 mg, 30 mg, 40 mg|
|Zidovudine|S, T|Tablets|100 mg, 300 mg|
|Stavudine+ Lamivudine|S, T|Tablets|30 mg + 150 mg|
|Zidovudine+ Lamivudine+ Nevirapine|S, T|Tablets|300 mg + 150 mg + 200 mg|
|Efavirenz|S, T|Capsules|200 mg, 600 mg|
|Nevirapine|S, T|Suspension|50 mg / 5 ml, 200 mg|
|Indinavir|S, T|Capsules|200 mg, 400 mg|
|Nelfinavir|S, T|Capsules|250 mg|
|Ritonavir|S, T|Syrup|100 mg, 400 mg / 5 ml|
|Saquinavir|S, T|Capsules|200 mg|
---
# 6.5 Antiprotozoal Medicines

# 6.5.1 Antiamoebic and Antigiardiasis medicines

|Medicines|Category|Route of Administration/Dosage Form|Strengths|
|---|---|---|---|
|Diloxanide Furoate|P,S,T|Tablets|500 mg|
|Metronidazole|P,S,T|Tablets|200 mg, 400 mg|
| | |Injection|500 mg / 100 ml|

# 6.5.2 Antileishmaniasis medicines

|Medicines|Category|Route of Administration/Dosage Form|Strengths|
|---|---|---|---|
|Amphotericin B|S, T|Injection|50 mg|
|Pentamidine Isothionate|S, T|Injection|200 mg|
|Sodium Stibogluconate|S, T|Injection|100 mg / ml|

# 6.5.3 Antimalarial Medicines

# 6.5.3.1 For curative treatment

|Medicines|Category|Route of Administration/Dosage Form|Strengths|
|---|---|---|---|
|Artesunate (To be used only in combination with Sulfadoxine + Pyrimethamine)|P,S,T|Tablets|50 mg|
|Chloroquine phosphate|P,S,T|Tablets|150 mg base|
| | |Injection|40 mg / ml|
| | |Syrup|50 mg / 5 ml|
|Primaquine|P,S,T|Tablets|2.5 mg, 7.5 mg|
|Pyrimethamine|P,S,T|Tablets|25 mg|
|Quinine sulphate|ST|Injection|300 mg / ml|
|Sulfadoxine + Pyrimethamine|P,S,T|Tablets|500 mg + 25 mg|
---
|Medicines|Category|Route of Administration/Dosage Form|Strengths|
|---|---|---|---|
|Clindamycin|S, T|Tablet|150, 300mg|
|Mefloquine|S, T|Tablet|250 mg base|
|Co-Trimoxazole (Trimethoprim + Sulphamethoxazole)|P, S, T|Tablets|80 + 400 mg, 160+800 mg|
| | |Suspension|40 + 200 mg / 5 ml|
|Pentamidine Isothionate|S, T|Injection|200 mg|
---
|Medicines|Category|Route of Administration/Dosage Form|Strengths|
|---|---|---|---|
|Acetyl Salicylic Acid|P, S, T|Tablets|300 - 350 mg|
|Dihydroergotamine|S, T|Tablets|1 mg|
|Paracetamol|P, S, T|Tablets|500 mg|

|Medicines|Category|Route of Administration/Dosage Form|Strengths|
|---|---|---|---|
|Propranolol hydrochloride|P, S, T|Tablets|10 mg, 40 mg|
---
|Medicines|Category|Route of Administration/Dosage Form|Strengths|
|---|---|---|---|
|Azathioprine|T|Tablets|50 mg|
|Cyclosporine|T|Capsules|10 mg, 25 mg, 50 mg, 100 mg|
|Cyclosporine|T|Concentrate for Injection|100 mg/ml|

|Medicines|Category|Route of Administration/Dosage Form|Strengths|
|---|---|---|---|
|Actinomycin D|T|Injection|0.5 mg|
|Alpha Interferon|T|Injection|3 million IU|
|Bleomycin|T|Injection|15 mg|
|Busulphan|T|Tablets|2 mg|
|Cisplatin|T|Injection|50 mg/vial|
|Cisplatin|T|Tablets|50 mg, 200 mg|
|Cyclophosphamide|T|Injection|500 mg|
|Cytosine arabinoside|T|Injection|500 mg/vial, 1000 mg/vial|
|Danazol|T|Capsules|50 mg, 100 mg|
|Doxorubicin|T|Injection|10 mg, 50 mg|
|Etoposide|T|Injection|100 mg/5 ml vial|
---
|Medicine|Type|Form|Strength|
|---|---|---|---|
|Flutamide|T|Tablet|250 mg|
|5-Fluorouracil|T|Injection|250 mg / 5 ml|
|Folinic Acid|T|Injection|3 mg / ml|
|Gemcitabine hydrochloride|T|Injection|200 mg|
|L-Asparaginase|T|Injection|5000 KU|
|Melphalan|T|Tablet|2 mg, 5 mg|
| | |Tablet|50 mg|
|Mercaptopurine|T|Injection|100 mg / ml|
| | |Tablet|2.5 mg|
|Methotrexate|T|Injection|50 mg / ml|
|Mitomycin-C|T|Injection|10 mg|
|Paclitaxel|T|Injection|30 mg / 5 ml|
|Procarbazine|T|Capsules|50 mg|
|Vinblastine sulphate|T|Injection|10 mg|
|Vincristine|T|Injection|1 mg / ml|
|Carboplatin|T|Injection|150 mg, 450 mg vial|
|Dacarbazine|T|Injection|500 mg|
|Daunorubicin|T|Injection|20 mg vial|
|Ifosfamide|T|Injection|1 gm/2ml vial|
|Mesna|T|Injection|200 mg|
|Oxaliplatin|T|Injection|50 mg vial|
|Imatinib|T|Tablets|100 mg, 400 mg|
|Chlorambucil|T|Tablets|2 mg|
---
# 8.3: Hormones and antihormones

|Medicines|Category|Route of Administration/Dosage Form|Strengths|
|---|---|---|---|
|Prednisolone|S, T|Injection|20 mg, 25 mg (as sodium phosphate or succinate)|
|Raloxifene|T|Tablets|60 mg|
|Tamoxifen Citrate|T|Tablets|10 mg, 20 mg|

# 8.4: Medicines used in palliative care

|Medicines|Category|Route of Administration/Dosage Form|Strengths|
|---|---|---|---|
|Morphine Sulphate|T|Tablets|10 mg|
|Morphine Sulphate| |Tablets|4 mg, 8 mg|
|Ondansetron|S, T|Injection|2 mg/ml|
|Ondansetron| |Syrup|2 mg/5 ml|

# Added Medicines

|Medicines|Category|Route of Administration/Dosage Form|Strengths|
|---|---|---|---|
|Filgrastim|T|Injection|1 ml vial|
|Allopurinol|T|Tablets|100 mg|
---
|Medicines|Category|Route of Administration/Dosage Form|Strengths|
|---|---|---|---|
|Bromocriptine Mesylate|S, T|Tablets|1.25 mg, 2.5 mg|
|Levodopa+ Carbidopa|P, S, T|Tablets|100 mg+10 mg, 250 mg+25 mg, 100 mg+25 mg|
|Trihexyphenidyl Hydrochloride|P, S, T|Tablets|2 mg|

Page 49 of 123
---
|Medicines|Category|Route of Administration/Dosage Form|Strengths|
|---|---|---|---|
|Cyanocobalamin|P, S, T|Injection|1 mg/ml|
|Ferrous Sulphate/Fumarate|P, S, T|Tablets|Tablets equivalent to 60 mg elemental iron|
| | |Oral solution|25mg elemental iron (as sulphate)/ml|
|Folic Acid|P, S, T|Tablets|1 mg, 5mg|
|Iron Dextran|S, T|Injection|50 mg iron/ml|
|Pyridoxine|P, S, T|Tablets|10 mg|

|Medicines|Category|Route of Administration/Dosage Form|Strengths|
|---|---|---|---|
|Heparin Sodium|S, T|Injection|5000 IU/ml|
|Protamine Sulphate|S, T|Injection|10 mg/ml|
|Phytomenadione|P, S, T|Injection|10 mg/ml|
|Warfarin sodium|S, T|Tablets|5 mg|
|Enoxaparin|T|Injection|40mg, 60mg|
---
|Medicines|Category|Route of Administration/Dosage Form|Strengths|
|---|---|---|---|
|Dextran-40|P, S, T|Injection|10%|
|Dextran-70|P, S, T|Injection|6%|
|Fresh frozen plasma|T|Injection| |
|Hydroxyethyl Starch (Hetastarch)|S, T|Injection|6%|
|Polygeline|S, T|Injection|3.5%|

|Medicines|Category|Route of Administration/Dosage Form|Strengths|
|---|---|---|---|
|Albumin|S, T|Injection|5%, 20 %|
|Cryoprecipitate|S, T|Injection| |
|Factor VIII Concentrate|S, T|Injection|Dried|
|Factor IX Complex (Coagulation Factors II, VII, IX, X)|S, T|Injection|Dried|
|Platelet Rich Plasma|S, T|Injection| |
---
|Medicines|Category|Route of Administration/Dosage Form|Strengths|
|---|---|---|---|
|Acetyl salicylic acid|P, S, T|Tablets soluble/dispersible|75mg, 100mg, 350 mg|
|Diltiazem|S, T|Tablets|30 mg, 60 mg|
|Glyceryl Trinitrate|P, S, T|Sublingual Tablets, Injection|0.5 mg, 5mg/ml|
|Isosorbide 5 Mononitrate/ Dinitrate|P, S, T|Tablets|10 mg, 20 mg|
|Metoprolol|P, S, T|Tablets, Injection|25 mg, 50 mg, 1mg/ml|
|Clopidogrel|T|Tablets|75 mg|
|Adenosine|S, T|Injection|3 mg/ml|
|Amiodarone|S, T|Tablets, Injection|100 mg, 200 mg, 50 mg/ml (3 ml ampoule)|
|Diltiazem|S, T|Tablets, Injection|30 mg, 60 mg, 5 mg/ml|
---
|Medicines|Category|Route of Administration/Dosage Form|Strengths|
|---|---|---|---|
|Amlodipine|P,S,T|Tablets|2.5 mg, 5 mg|
|Atenolol|P,S,T|Tablets|50 mg, 100 mg|
|Enalapril|P,S,T|Tablets|2.5 mg, 5 mg|
|Enalapril Maleate|T|Injection|1.25 mg/ml|
|Losartan Potassium|S, T|Tablets|25 mg, 50 mg|
|Methyldopa|P,S, T|Tablets|250 mg|
|Nifedipine|S, T|Tablets|5 mg, 10 mg, 10 mg, 20 mg, 10 mg, 20 mg|
|Sodium Nitroprusside|T|Injection|50 mg/5 ml|

|Added Medicines| |
|---|---|
|Hydrochlorthiazide|P,S,T|Tablets|12.5 mg, 25 mg|
---
# 12.4: Medicines used in heart failure

|Medicines|Category|Route of Administration/Dosage Form|Strengths|
|---|---|---|---|
|Digoxin|S, T|Tablets|0.25 mg|
|Digoxin|S, T|Injection|0.25 mg/ml|
|Digoxin|S, T|Elixir|0.05 mg/ml|
|Dobutamine|S, T|Injection|50 mg/ml|
|Dopamine Hydrochloride|S, T|Injection|40 mg/ml|

# 12.5: Antithrombotic medicines

|Medicines|Category|Route of Administration/Dosage Form|Strengths|
|---|---|---|---|
|Acetyl salicylic acid|P, S, T|Tablets|75mg, 100mg, 350 mg soluble/dispersible|
|Heparin Sodium|S, T|Injection|1000 IU/ml, 5000 IU/ml|
|Streptokinase|S, T|Injection|750,000 IU, 15,00,000 IU|
|Urokinase|T|Injection|500,000 IU/ml, 10,00,000 IU/ml|

# New Category - ADDED

# 12.6 Hypolipidemic Medicines

|Medicines|Category|Route of Administration/Dosage Form|Strengths|
|---|---|---|---|
|Atorvastatin|P, S, T|Tablets|5 mg, 10 mg|

Page 54 of 123
---
|Medicines|Category|Route of Administration/Dosage Form|Strengths|
|---|---|---|---|
|Miconazole|P,S,T|Ointment or Cream|2%|
|Acyclovir|S, T|Cream|5%|
|Framycetin Sulphate|P,S,T|Cream|0.5%|
|Methylrosanilinium Chloride (Gentian Violet)|P,S,T|Aqueous solution|0.5%|
|Neomycin + Bacitracin|P,S,T|Ointment|5 mg + 500 IU / g|
|Povidone Iodine|P,S,T|Solution or Ointment|5%|
|Silver Sulphadiazine|P,S,T|Cream|1%|
|Betamethasone Dipropionate|P,S,T|Cream / Ointment|0.05%|
|Calamine|P,S,T|Lotion| |
|Zinc Oxide|P,S,T|Dusting Powder| |
---
|Medicines|Category|Route of Administration/Dosage Form|Strengths|
|---|---|---|---|
|Coal Tar|P,S,T|Solution|5%|
|Dithranol|T|Ointment|0.1-2%|
|Glycerin|P,S,T|Solution| |
|Salicylic Acid|P,S,T|Solution|5%|

|Medicines|Category|Route of Administration/Dosage Form|Strengths|
|---|---|---|---|
|Benzyl benzoate|P,S,T|Lotion|25%|
|Permethrin|S,T|Lotion|1%, 5%|
---
|Medicines|Category|Route of Administration/Dosage Form|Strengths|
|---|---|---|---|
|Fluorescein|S, T|Eye drops|1%|
|Lignocaine|S, T|Eye Drops|4%|
|Tropicamide|S, T|Eye drops|1%|

|Medicines|Category|Route of Administration/Dosage Form|Strengths|
|---|---|---|---|
|Barium Sulphate|S, T|Suspension|100% w/v, 250% w/v|
|Calcium Ipodate|S, T|Injection|3 g|
|Iopanoic Acid|S, T|Tablets|500 mg|
|Meglumine Iothalamate|S, T|Injection|60% w/v (iodine = 280 mg/ml)|
|Meglumine Iotroxate|S, T|Solution|5-8 g iodine in 100-250 ml|
|Propyliodone|S, T|Oily, suspension|500-600 mg/ml|
|Sodium Iothalamate|S, T|Injection|70% w/v (Iodine = 420 mg/ml)|
|Sodium Meglumine Diatrizoate|S, T|Injection|60% w/v (Iodine conc. = 292 mg/ml), 76% w/v (Iodine conc. = 370 mg/ml)|
---
|Medicines|Category|Route of Administration/Dosage Form|Strengths|
|---|---|---|---|
|Acriflavin+Glycerin|P, S, T|Solution| |
|Benzoin Compound|P, S, T|Tincture| |
|Cetrimide|P, S, T|Solution|20% (conc. for dilution)|
|Chlorhexidine|P, S, T|Solution|5% (conc. for dilution)|
|Ethyl Alcohol 70%|P, S, T|Solution| |
|Gentian Violet|P, S, T|Paint|0.5%, 1%|
|Hydrogen Peroxide|P, S, T|Solution|6%|
|Povidone Iodine|P, S, T|Solution|5%, 10%|

|Medicines|Category|Route of Administration/Dosage Form|Strengths|
|---|---|---|---|
|Bleaching Powder|P, S, T|Powder|Contains not less than 30% w/w of available chlorine (as per I.P)|
|Formaldehyde Solution|P, S, T|Solution|Dilute 34 ml of formaldehyde solution with water to produce 100 ml (As per I.P)|
|Glutaraldehyde|S, T|Solution|2%|
|Potassium Permanganate|P, S, T|Crystals for solution| |
---
|Medicines|Category|Route of Administration/Dosage Form|Strengths|
|---|---|---|---|
|Furosemide|P,S,T|Tablets|40mg|
|Hydrochlorothiazide|P,S,T|Tablets|25 mg, 50 mg|
|Mannitol|P,S,T|Injection|10%, 20%|
|Spironolactone|P,S,T|Tablets|25 mg|
---
# Section: 17 – Gastrointestinal medicines

# 17.1: Antacids and other Antiulcer medicines

|Medicines|Category|Route of Administration/Dosage Form|Strengths|
|---|---|---|---|
|Aluminium Hydroxide + Magnesium Hydroxide|P,S,T|Tablet, Suspension| |
|Omeprazole|P,S,T|Capsules|10 mg, 20 mg, 40 mg|
|Ranitidine|P,S,T|Injection|25 mg / ml|

# Added Medicines

|Pantoprazole|T|Injection|40 mg|
|---|---|---|---|
|Famotidine|P,S,T|Tablets|20 mg|
|# 17.2: Antiemetics<br/><br/>Medicines|Category|Route of Administration/Dosage Form|Strengths|
|Domperidone|P,S,T|Tablets, Syrup|10 mg, 1 mg / ml|
|Metoclopramide|P,S,T|Tablets, Syrup, Injection|10 mg, 5 mg / 5 ml, 5 mg / ml|
|Promethazine|P,S,T|Tablets, Elixir or Syrup, Injection|10 mg, 25 mg, 5 mg / 5 ml, 25 mg / ml|

# Added Medicines

Ondansetron
S,T
Tablet, Syrup, Injection
4 mg, 8 mg, 2 mg/ml, 2 mg/ml

Page 60 of 123
---
|Medicines|Category|Route of Administration/Dosage Form|Strengths|
|---|---|---|---|
|5-Amino salicylic Acid (5-ASA)|S,T|Tablets|400mg|

|Medicines|Category|Route of Administration/Dosage Form|Strengths|
|---|---|---|---|
|Dicyclomine Hydrochloride|P,S,T|Tablets, Injection|10 mg, 10 mg / ml|
|Hyoscine Butyl Bromide|P,S,T|Tablets, Injection|10 mg, 20 mg / ml|

|Medicines|Category|Route of Administration/Dosage Form|Strengths|
|---|---|---|---|
|Bisacodyl|P,S,T|Tablets, Suppository|5 mg|
|Ispaghula|P,S,T|Granules| |

|Medicines|Category|Route of Administration/Dosage Form|Strengths|
|---|---|---|---|
|Oral Rehydration Salts|P,S,T|Powder for solution|Glucose: 13.5 g/L, Sodium chloride: 2.6 g/L, Potassium chloride: 1.5 g/L, Trisodium citrate dihydrate+: 2.9 g/L, Powder for dilution in 200ml; 500 ml; 1000ml. (As per I.P)|
---
|Medicines|Category|Route of Administration/Dosage Form|Strengths|
|---|---|---|---|
|Zinc Sulfate|P,S,T|Syrup|20 mg/5ml|

Page 62 of 123
---
Section: 18 – Hormones, oper endocrine medicines and contraceptives

18.1: Adrenal hormones and synpetic substitutes

|Medicines|Category|Route of Administration/Dosage Form|Strengths|
|---|---|---|---|
|Dexamethasone|S, T|Tablets|0.5mg|
|Dexamethasone|S, T|Injection|4mg/ml|
|Hydrocortisone Sodium Succinate|P, S, T|Injection|100 mg/ml|
|Methyl Prednisolone|S, T|Injection|40 mg/ml|
|Prednisolone|P, S, T|Tablets|5mg, 10mg, 20mg|

18.2: Androgens

|Medicines|Category|Route of Administration/Dosage Form|Strengths|
|---|---|---|---|
|Testosterone|T|Capsules|40mg (as undecanoate)|
|Testosterone|T|Injection|25mg/ml (as propionate)|

18.3: Contraceptives

18.3.1: Hormonal Contraceptives

|Medicines|Category|Route of Administration/Dosage Form|Strengths|
|---|---|---|---|
|Ethinylestradiol + Levonorgestrel|P, S, T|Tablets|0.03 mg + 0.15 mg|
|Ethinylestradiol + Norethisterone|P, S, T|Tablets|0.035 mg + 1.0 mg|
|Hormone Releasing IUD|T|Levonorgestrel Releasing IUD| |
---
|Medicines|Category|Route of Administration/Dosage Form|Strengths|
|---|---|---|---|
|IUD containing Copper|P,S,T| | |

|Medicines|Category|Route of Administration/Dosage Form|Strengths|
|---|---|---|---|
|Condoms|P,S,T| | |

|Medicines|Category|Route of Administration/Dosage Form|Strengths|
|---|---|---|---|
|Ethinylestradiol|P,S,T|Tablets|0.01mg, 0.05mg|

**18.5: Medicines used in Diabetes mellitus**
|Medicines|Category|Route of Administration/Dosage Form|Strengths|
|---|---|---|---|
|Glibenclamide|P,S,T|Tablets|2.5 mg, 5mg|
|Insulin Injection (Soluble)|P,S,T|Injection|40 IU/ml|
|Intermediate Acting(Lente/NPH Insulin)|P,S,T|Injection|40 IU/ml|
|Metformin|P,S,T|Tablets|500mg|

|Medicines|Category|Route of Administration/Dosage Form|Strengths|
|---|---|---|---|
|Premix Insulin 30:70 injection|P,S,T|Injection|40IU/ml|
---
|Medicines|Category|Route of Administration/Dosage Form|Strengths|
|---|---|---|---|
|Glucagon|T|Injection|1mg/ml|
|25% Dextrose|P,S,T|Injection|100 ml|

|Medicines|Category|Route of Administration/Dosage Form|Strengths|
|---|---|---|---|
|Clomiphene citrate|T|Tablets|50mg, 100mg|

|Medicines|Category|Route of Administration/Dosage Form|Strengths|
|---|---|---|---|
|Medroxy Progesterone Acetate|P,S,T|Tablets|5mg, 10mg|
|Norethisterone|P,S,T|Tablets|5mg|

|Medicines|Category|Route of Administration/Dosage Form|Strengths|
|---|---|---|---|
|Carbimazole|P,S,T|Tablets|5mg, 10mg|
|Levothyroxine|P,S,T|Tablets|50μg, 100μg|
|Iodine|S,T|Solution|8 mg / 5 ml|
---
|Drugs|Category|Route of Administration/Dosage Form|Strengths|
|---|---|---|---|
|Tuberculin, Purified Protein derivative|P,S,T|Injection|1 TU, 5 TU|

|Drugs|Category|Route of Administration/Dosage Form|Strengths|
|---|---|---|---|
|Anti-D immunoglobin (human)|S, T|Injection|300 μg|
|Polyvalent Antisnake Venom|P,S,T|Injection|10 ml|
|Antitetanus Human immunoglobin|P,S,T|Injection|250 IU, 500 IU|
|Diphtheria Antitoxin|S, T|Injection|10,000 IU|
|Rabies immunoglobin|P,S,T|Injection|150 IU / ml|

|Drugs|Category|Route of Administration/Dosage Form|Strengths|
|---|---|---|---|
|B.C.G Vaccine|P,S,T|Injection| |
|D.P.T Vaccine|P,S,T|Injection| |
|Hepatitis B Vaccine|P,S,T|Injection| |
---
|Drugs|Category|Route of Administration/Dosage Form|Strengths|
|---|---|---|---|
|Measles Vaccine|P,S,T|Injection| |
|Oral Poliomyelitis vaccine (LA)|P,S,T|Solution| |
|Rabies Vaccine|P,S,T|Injection| |
|Tetanus Toxoid|P,S,T|Injection| |
---
|Drugs|Category|Route of Administration/Dosage Form|Strengths|
|---|---|---|---|
|Atracurium besylate|S, T|Injection|10 mg / ml|
|Neostigmine|S, T|Tablets, Injection|15 mg, 0.5mg/ml|
|Pyridostigmine|S, T|Tablets, Injection|60 mg, 1mg/ml|
|Succinyl choline chloride|S, T|Injection|50 mg/ml|
|Vecuronium|P, S, T|Injection|2 mg/ml|
---
# Section: 21 – Ophthalmological Preparations

|Medicines|Category|Route of Administration/Dosage Form|Strengths|
|---|---|---|---|
|Chloramphenicol|P,S,T|Drops/Ointment|0.4%, 1%|
|Ciprofloxacin Hydrochloride|P,S,T|Drops/Ointment|0.3%|
|Gentamicin|P,S,T|Drops|0.3%|
|Miconazole|P,S,T|Drops|1%|
|Povidone Iodine|S,T|Drops|0.6%|
|Sulphacetamide Sodium|P,S,T|Drops|10%, 20%|

# 21.2: Antiinflammaory agents

|Medicines|Category|Route of Administration/Dosage Form|Strengths|
|---|---|---|---|
|Prednisolone Acetate|P,S,T|Drops|0.1%|
|Prednisolone Sodium Phosphate|P,S,T|Drops|1%|

# 21.3: Local Anaesthetics

|Medicines|Category|Route of Administration/Dosage Form|Strengths|
|---|---|---|---|
|Tetracaine Hydrochloride|P,S,T|Drops|0.5%|

# 21.4: Miotics and Antiglucoma medicines

|Medicines|Category|Route of Administration/Dosage Form|Strengths|
|---|---|---|---|
| | | | |
---
|Medicines|Category|Route of Administration/Dosage Form|Strengths|
|---|---|---|---|
|Acetazolamide|S, T|Tablets|250 mg|
|Betaxolol Hydrochloride|T|Drops|0.25%|
|Pilocarpine|S, T|Drops|2%|
|Timolol Maleate|P, S, T|Drops|0.5%|

|Medicines|Category|Route of Administration/Dosage Form|Strengths|
|---|---|---|---|
|Atropine Sulphate|P, S, T|Drops/Ointment|1%|
|Homatropine|P, S, T|Drops|2%|
|Phenylephrine|P, S, T|Drops|5%|

|Medicines|Category|Route of Administration/Dosage Form|Strengths|
|---|---|---|---|
|Methyl Cellulose|T|Injection|2%|

|Medicines|Category|Route of Administration/Dosage Form|Strengths|
|---|---|---|---|
|Methyl Ergometrine|P, S, T|Injection|0.2mg/ml|
|Mifepristone|T|Tablets|200mg|
|Oxytocin|S, T|Injection|5 IU/ml, 10 IU/ml|
---
| | |Added medicines| |
|---|---|---|---|
|Misoprostol|T|Tablets|100μg|
| | |Antioxytocics| |
|Medicines|Category|Route of Administration/ Dosage Form|Strengths|
|Terbutaline Sulphate|S,T|Tablets|2.5 mg|
| | |Injection|0.5 mg/ml|
| | |Added Medicines| |
|Nifedipine|S,T|Tablets|10 mg|
|Betamethasone|P,S,T|Injection|4 mg/ml|

Page 71 of 123
---
# Section: 23 – Peritoneal Dialysis Solution

|Medicines|Category|Route of Administration/Dosage Form|Strengths|
|---|---|---|---|
|Intraperitoneal Dialysis Solution|T| |4L of approximate composition|

# Section: 24 – Psychotherapeutic Medicines

# 24.1: Medicines used in Psychotic Disorders

|Medicines|Category|Route of Administration/Dosage Form|Strengths|
|---|---|---|---|
|Chlorpromazine hydrochloride|P,S,T|Tablets|25 mg, 50 mg, 100 mg|
|Chlorpromazine hydrochloride|P,S,T|Syrup|25 mg/5ml|
|Chlorpromazine hydrochloride|P,S,T|Injection|25 mg/ml|
|Haloperidol|S, T|Injection|5 mg/ml|
|Olanzapine|T|Tablets|5 mg, 10 mg|

# 24.2: Medicines used in mood disorders

# 24.2.1: Medicines used in Depressive disorders

|Medicines|Category|Route of Administration/Dosage Form|Strengths|
|---|---|---|---|
|Amitriptyline|P,S,T|Tablets|25 mg|
|Fluoxetine hydrochloride|P,S,T|Capsules|20 mg|
|Imipramine|P,S,T|Tablets|25 mg, 75 mg|

# 24.2.2: Medicines used in Bipolar disorders

|Medicines|Category|Route of Administration/Dosage Form|Strengths|
|---|---|---|---|
|Lithium Carbonate|T|Tablets|300 mg|

Page 72 of 123
---
# Added Medicines

|Sodium Valproate|Category|Route of Administration/Dosage Form|Strengths|
|---|---|---|---|
|Tablets|P,S,T| |200 mg, 500mg|

# Medicines used for Generalized Anxiety and Sleep Disorders

|Medicines|Category|Route of Administration/Dosage Form|Strengths|
|---|---|---|---|
|Alprazolam|P,S,T|Tablets|0.25 mg, 0.5 mg|
|Diazepam|P,S,T|Tablets|2 mg, 5mg|

# Medicines used for obsessive compulsive disorders and panic attacks

|Medicines|Category|Route of Administration/Dosage Form|Strengths|
|---|---|---|---|
|Fluoxetine hydrochloride|P,S,T|Capsules|20 mg|
---
|Medicines|Category|Route of Administration/Dosage Form|Strengths|
|---|---|---|---|
|Beclomethasone Dipropionate|P,S,T|Inhalation|50 μg, 250μg/dose|
|Hydrocortisone sodium succinate|P,S,T|Injection|100 mg, 200mg, 400 mg|
|Salbutamol sulphate|P,S,T|Syrup|2mg/5ml|
| | |Inhalation|100μg/dose|
|Ipratropium bromide|P,S,T|Inhalation|20μg/metered dose|

|Medicines|Category|Route of Administration/Dosage Form|Strengths|
|---|---|---|---|
|Codeine phosphate|S,T|Tablets|10mg|
| | |Syrup|15mg/5ml|
|Dextromethorphan|P,S,T|Tablets|30mg|
---
Section: 26 – Solutions correcting water, electrolyte and acid-base disturbances

26.1: Oral

|Medicines|Category|Route of Administration/Dosage Form|Strengths|
|---|---|---|---|
|Oral Rehydration Salts|P, S, T|Powder for Solution|As per IP|

26.2: Parenteral

|Medicines|Category|Route of Administration/Dosage Form|Strengths|
|---|---|---|---|
|Glucose|P, S, T|Injection|5% isotonic, 10%, 15%|
|Glucose with sodium chloride|P, S, T|Injection|5% + 0.9%|
|Normal Saline|P, S, T|Injection|0.9%|
|N/2 Saline|S, T|Injection| |
|N/5 Saline|S, T|Injection| |
|Potassium Chloride|P, S, T|Injection|11.2% Sol.|
|Ringer Lactate|P, S, T|Injection|As per IP|
|Sodium Bicarbonate|P, S, T|Injection|As per IP|

26.3: Miscellaneous

|Medicines|Category|Route of Administration/Dosage Form|Strengths|
|---|---|---|---|
|Water for Injection|P, S, T|Injection|2 ml, 5 ml, 10 ml|

Page 75 of 123
---
|Medicines|Category|Route of Administration/Dosage Form|Strengths|
|---|---|---|---|
|Ascorbic Acid|P,S,T|Tablets|100 mg, 500 mg|
|Calcium carbonate|P,S,T|Tablets|250 mg, 500 mg|
|Multivitamins (As per Schedule V of Drugs and Cosmetics Rules)|P,S,T|Tablets| |
|Nicotinamide|P,S,T|Tablets|50 mg|
|Pyridoxine|P,S,T|Tablets|25 mg|
|Riboflavin|P,S,T|Tablets|5 mg|
|Thiamine|P,S,T|Tablets|100 mg|
|Vitamin A|P,S,T|Capsules, Injection|5000 IU, 50000 IU, 100000 IU, 50000 IU/ml|
|Vitamin D (Ergocalciferol)|P,S,T|Capsules|0.25 mg, 1 mg|
|Calcium gluconate|P,S,T|Injection|100mg/ml in 10 ml ampoule|
---
# Alphabetical List of Medicines – Therapeutic area wise

|S.No|Section 1: – Anesthesia|
|---|---|
|1.|Atropine Sulphate|
|2.|Bupivacaine Hydrochloride|
|3.|Diazepam|
|4.|EMLA cream|
|5.|Ether|
|6.|Halothane with vaporizer|
|7.|Isoflurane|
|8.|Ketamine Hydrochloride|
|9.|Lignocaine Hydrochloride|
|10.|Lignocaine Hydrochloride + Adrenaline|
|11.|Midazolam|
|12.|Morphine Sulphate|
|13.|Nitrous Oxide|
|14.|Oxygen|
|15.|Promethazine|
|16.|Propofol|
|17.|Sevoflurane|
|18.|Thiopentone Sodium|

|S.No|Section 2 – Analgesics, Antipyretics, Nonsteroidal anti-inflammatory medicines, Medicines used to treat Gout and Disease Modifying Agents used in Rheumatoid Disorders|
|---|---|
|1.|Acetyl Salicylic Acid|
|2.|Allopurinol|
|3.|Azathioprine|
|4.|Colchicine|
---
# Alphabetical List of Medicines – Therapeutic area wise

|5.|Diclofenac|
|---|---|
|6.|Fentanyl|
|7.|Hydroxychloroquine phosphate|
|8.|Ibuprofen|
|9.|Leflunomide|
|10.|Methotrexate|
|11.|Morphine Sulphate|
|12.|Paracetamol|
|13.|Sulfasalazine|
|14.|Tramadol|

# SECTION: 3 – Antiallergics and Medicines used in Anaphylaxis

|1.|Adrenaline Bitartrate|
|---|---|
|2.|Cetrizine|
|3.|Chlorpheniramine Maleate|
|4.|Dexamethasone|
|5.|Dexchlorpheniramine Maleate|
|6.|Hydrocortisone Sodium Succinate|
|7.|Pheniramine Maleate|
|8.|Prednisolone|
|9.|Promethazine|

# SECTION: 4 – Antidotes and Other Substances used in Poisonings

|1.|Activated Charcoal|
|---|---|
|2.|Atropine Sulphate|
|3.|Calcium gluconate|
|4.|Desferrioxamine mesylate|
---
# Alphabetical List of Medicines – Therapeutic area wise

|5.|Dimercaprol|
|---|---|
|6.|Flumazenil|
|7.|Methylthioninium chloride (Methylene blue)|
|8.|N-acetylcysteine|
|9.|Naloxone|
|10.|Penicillamine|
|11.|Pralidoxime Chloride(2-PAM)|
|12.|Sodium Nitrite|
|13.|Sodium Thiosulphate|
|14.|Specific antisnake venom|

# SECTION: 5 – Anticonvulsants/ Antiepileptics

|1.|Carbamazepine|
|---|---|
|2.|Diazepam|
|3.|Lorazepam|
|4.|Magnesium sulphate|
|5.|Phenobarbitone|
|6.|Phenytoin Sodium|
|7.|Sodium Valproate|

# Section:6 – Anti-infective Medicines

|1.|Acyclovir|
|---|---|
|2.|Albendazole|
|3.|Amikacin|
|4.|Amoxicillin|
|5.|Amoxicillin+Clavulinic acid|
|6.|Amphotericin B|
---
|7.|Ampicillin|
|---|---|
|8.|Artesunate (To be used only in combination with Sulfadoxine + Pyrimethamine)|
|9.|Azithromycin|
|10.|Benzathine Benzylpenicillin|
|11.|Cefixime|
|12.|Cefotaxime|
|13.|Ceftazidime|
|14.|Ceftriaxone|
|15.|Cephalexin|
|16.|Chloroquine phosphate|
|17.|Ciprofloxacin Hydrochloride|
|18.|Clindamycin|
|19.|Clofazimine|
|20.|Clotrimazole|
|21.|Cloxacillin|
|22.|Co-trimoxazole (Trimethoprim + Sulphamethoxazole)|
|23.|Dapsone|
|24.|Didanosine|
|25.|Diethylcarbamazine citrate|
|26.|Diloxanide Furoate|
|27.|Doxycycline|
|28.|Efavirenz|
|29.|Erythromycin Estolate|
|30.|Ethambutol|
|31.|Fluconazole|

Page 80 of 123
---
# Alphabetical List of Medicines – Therapeutic area wise

32. Gentamicin
33. Griseofulvin
34. Indinavir
35. Isoniazid
36. Lamivudine
37. Lamivudine + Nevirapine + Stavudine
38. Lamivudine + Zidovudine
39. Mefloquine
40. Metronidazole
41. Nelfinavir
42. Nevirapine
43. Nitrofurantoin
44. Nystatin
45. Ofloxacin
46. Pentamidine Isopionate
47. Piperazine
48. Praziquantel
49. Primaquine
50. Pyrazinamide
51. Pyrimepamine
52. Quinine sulphate
53. Rifampicin
54. Ritonavir
55. Saquinavir
56. Sodium Stibogluconate

Page 81 of 123
---
# Alphabetical List of Medicines – Therapeutic area wise

57. Stavudine
58. Stavudine+Lamivudine
59. Streptomycin Sulphate
60. Sulfadoxine + Pyrimepamine
61. Sulphadiazine
62. Vancomycin Hydrochloride
63. Zidovudine
64. Zidovudine+ Lamivudine+ Nevirapine

# SECTION: 7 –Antimigraine medicines

1. Acetyl Salicylic Acid
2. Dihydroergotamine
3. Paracetamol
4. Propranolol hydrochloride

# SECTION: 8 –Antineoplastic, immunosuppressives and medicines used in palliative care

1. 5-Fluorouracil
2. Actinomycin D
3. Allopurinol
4. Alpha Interferon
5. Azapioprine
6. Bleomycin
7. Busulphan
8. Carboplatin
9. Chlorambucil
10. Cisplatin
11. Cyclophosphamide
---
# Alphabetical List of Medicines – Therapeutic area wise

12. Cyclosporine
13. Cytosine arabinoside
14. Dacarbazine
15. Danazol
16. Daunorubicin
17. Doxorubicin
18. Etoposide
19. Filgrastim
20. Flutamide
21. Folinic Acid
22. Gemcitabine hydrochloride
23. Ifosfamide
24. Imatinib
25. L- Asparaginase
26. Melphalan
27. Mercaptopurine
28. Mesna
29. Mepotrexate
30. Mitomycin-C
31. Morphine Sulphate
32. Ondansetron
33. Oxaliplatin
34. Paclitaxel
35. Prednisolone
36. Procarbazine

Page 83 of 123
---
# Alphabetical List of Medicines – Therapeutic area wise

|37.|Raloxifene|
|---|---|
|38.|Tamoxifen Citrate|
|39.|Vinblastine sulphate|
|40.|Vincristine|

# SECTION: 9 –Antiparkinsonism medicines

|1.|Bromocriptine Mesylate|
|---|---|
|2.|Levodopa+ Carbidopa|
|3.|Trihexyphenidyl Hydrochloride|

# SECTION: 10 –Medicines affecting the blood

|1.|Cyanocobalamin|
|---|---|
|2.|Enoxaparin|
|3.|Ferrous Salt|
|4.|Folic Acid|
|5.|Heparin Sodium|
|6.|Iron Dextran|
|7.|Phytomenadione|
|8.|Protamine Sulphate|
|9.|Pyridoxine|
|10.|Warfarin sodium|

# SECTION: 11 –Blood products and Plasma substitutes

|1.|Albumin|
|---|---|
|2.|Cryoprecipitate|
|3.|Dextran-40|
|4.|Dextran-70|
|5.|Factor IX Complex (Coagulation Factors II,VII, IX, X)|
---
# Alphabetical List of Medicines – Therapeutic area wise

|6.|Factor VIII Concentrate|
|---|---|
|7.|Fresh frozen plasma|
|8.|Hydroxyethyl Starch (Hetastarch)|
|9.|Platelet Rich Plasma|
|10.|Polygeline|

# SECTION: 12 –Cardiovascular medicines

|1.|Acetyl salicylic acid|
|---|---|
|2.|Adenosine|
|3.|Amiodarone|
|4.|Amlodipine|
|5.|Atenolol|
|6.|Atorvastatin|
|7.|Clopidogrel|
|8.|Digoxin|
|9.|Diltiazem|
|10.|Dobutamine|
|11.|Dopamine Hydrochloride|
|12.|Enalapril Maleate|
|13.|Esmolol|
|14.|Glyceryl Trinitrate|
|15.|Heparin Sodium|
|16.|Hydrochlorthiazide|
|17.|Isosorbide 5 Mononitrate/Dinitrate|
|18.|Lignocaine Hydrochloride|
|19.|Losartan Potassium|

Page 85 of 123
---
# Alphabetical List of Medicines – Therapeutic area wise

|20.|Methyldopa|
|---|---|
|21.|Metoprolol|
|22.|Nifedipine|
|23.|Procainamide Hydrochloride|
|24.|Sodium Nitroprusside|
|25.|Streptokinase|
|26.|Urokinase|
|27.|Verapamil|

# SECTION: 13 –Dermatological medicines (Topical)

|1.|Acyclovir|
|---|---|
|2.|Benzyl benzoate|
|3.|Betamethasone Dipropionate|
|4.|Calamine|
|5.|Coal Tar|
|6.|Dithranol|
|7.|Framycetin Sulphate|
|8.|Glycerin|
|9.|Methylrosanilinium Chloride (Gentian Violet)|
|10.|Miconazole|
|11.|Neomycin + Bacitracin|
|12.|Permethrin|
|13.|Povidone Iodine|
|14.|Salicylic Acid|
|15.|Silver Sulphadiazine|
|16.|Zinc Oxide|

Page 86 of 123
---
# Alphabetical List of Medicines – Therapeutic area wise

SECTION: 14 –Diagnostic agents
1. Barium Sulphate
2. Calcium Ipodate
3. Fluorescein
4. Iopanoic Acid
5. Lignocaine
6. Meglumine Iopalamate
7. Meglumine Iotroxate
8. Propyliodone
9. Sodium Iopalamate
10. Sodium Meglumine Diatrizoate
11. Tropicamide

SECTION: 15 –Disinfectants and antiseptics
1. Acriflavin+Glycerin
2. Benzoin Compound
3. Bleaching Powder
4. Cetrimide
5. Chlorhexidine
6. Epyl Alcohol 70%
7. Formaldehyde IP
8. Gentian Violet
9. Glutaraldehyde
10. Hydrogen Peroxide
11. Potassium Permanganate
12. Povidone Iodine

Page 87 of 123
---
SECTION: 16 – Diuretics
1. Furosemide
2. Hydrochloropiazide
3. Mannitol
4. Spironolactone

SECTION: 17 – Gastrointestinal medicines
1. 5-Amino salicylic Acid
2. Aluminium Hydroxide + Magnesium Hydroxide
3. Bisacodyl
4. Dicyclomine Hydrochloride
5. Domperidone
6. Famotidine
7. Hyoscine Butyl Bromide
8. Ispaghula
9. Metoclopramide
10. Omeprazole
11. Ondansetron
12. Oral Rehydration Salts
13. Pantoprazole
14. Promepazine
15. Ranitidine
16. Zinc Sulfate

SECTION: 18 – Hormones, oper endocrine medicines and contraceptives
1. 25% Dextrose
---
# Alphabetical List of Medicines – Therapeutic area wise

|2.|Carbimazole|
|---|---|
|3.|Clomiphene citrate|
|4.|Condoms|
|5.|Dexamethasone|
|6.|Ethinylestradiol|
|7.|Ethinylestradiol + Levonorgesterol|
|8.|Ethinylestradiol + Norethisterone|
|9.|Glibenclamide|
|10.|Glucagon|
|11.|Hormone Releasing IUD|
|12.|Hydrocortisone Sodium Succinate|
|13.|Insulin Injection(Soluble)|
|14.|Intermediate Acting(Lente/NPH Insulin)|
|15.|Iodine|
|16.|IUD containing Copper|
|17.|Levothyroxine|
|18.|Medroxy Progesterone Acetate|
|19.|Metformin|
|20.|MethylPrednisolone|
|21.|Norethisterone|
|22.|Prednisolone|
|23.|Premix Insulin 30:70 injection|
|24.|Testosterone|

# SECTION: 19 - Immunologicals

1.
Anti-D immunoglobin (human)
---
# Alphabetical List of Medicines – Therapeutic area wise

|2.|Antitetanus Human immunoglobin|
|---|---|
|3.|B.C.G Vaccine|
|4.|D.P.T Vaccine|
|5.|Diphtheria Antitoxin|
|6.|Hepatitis B Vaccine|
|7.|Measles Vaccine|
|8.|Oral Poliomyelitis vaccine (LA)|
|9.|Polyvalent Antisnake Venom|
|10.|Rabies|
|11.|Rabies Vaccine|
|12.|Tetanus Toxoid|
|13.|Tuberculin, Purified Protein derivative|

# SECTION: 20 – Muscle Relaxants (Peripherally acting) and Cholinesterase Inhibitors

|1.|Atracurium besylate|
|---|---|
|2.|Neostigmine|
|3.|Pyridostigmine|
|4.|Succinyl choline chloride|
|5.|Vecuronium|

# SECTION: 21 – Ophthalmological Preparations

|1.|Acetazolamide|
|---|---|
|2.|Atropine Sulphate|
|3.|Betaxolol Hydrochloride|
|4.|Chloramphenicol|
|5.|Ciprofloxacin Hydrochloride|
|6.|Gentamicin|

Page 90 of 123
---
# Alphabetical List of Medicines – Therapeutic area wise

|7.|Homatropine|
|---|---|
|8.|Methyl Cellulose|
|9.|Miconazole|
|10.|Phenylephrine|
|11.|Pilocarpine|
|12.|Povidone Iodine|
|13.|Prednisolone Acetate|
|14.|Prednisolone Sodium Phosphate|
|15.|Sulphacetamide Sodium|
|16.|Tetracaine Hydrochloride|
|17.|Timolol Maleate|

# SECTION: 22 – Oxytocics and Antioxytocics

|1.|Betamethasone|
|---|---|
|2.|Methyl Ergometrine|
|3.|Mifepristone|
|4.|Misoprostol|
|5.|Nifedipine|
|6.|Oxytocin|
|7.|Terbutaline Sulphate|

# SECTION: 23 – Peritoneal Dialysis Solution

1
Intraperitoneal Dialysis Solution

# SECTION: 24 – Psychotherapeutic Medicines

|1.|Alprazolam|
|---|---|
|2.|Amitriptyline|
|3.|Chlorpromazine hydrochloride|

Page 91 of 123
---
# Alphabetical List of Medicines – Therapeutic area wise

|4.|Diazepam|
|---|---|
|5.|Fluoxetine hydrochloride|
|6.|Haloperidol|
|7.|Imipramine|
|8.|Lithium Carbonate|
|9.|Olanzapine|
|10.|Sodium Valproate|

# SECTION: 25 – Medicines acting on the respiratory tract

|1.|Beclomethasone Dipropionate|
|---|---|
|2.|Codeine phosphate|
|3.|Dextromethorphan|
|4.|Hydrocortisone sodium succinate|
|5.|Ipratropium bromide|
|6.|Salbutamol sulphate|

# SECTION: 26 – Solutions correcting water, electrolyte and acid-base disturbances

|1.|Glucose|
|---|---|
|2.|Glucose with sodium chloride|
|3.|N/2 Saline|
|4.|N/5 Saline|
|5.|Normal Saline|
|6.|Oral Rehydration Salts|
|7.|Potassium Chloride|
|8.|Ringer Lactate|
|9.|Sodium Bicarbonate|
|10.|Water for Injection|

Page 92 of 123
---
# Alphabetical List of Medicines – Therapeutic area wise

SECTION: 27 – Vitamins and Minerals
1. Ascorbic Acid
2. Calcium gluconate
3. Calcium carbonate
4. Multivitamins (As per Schedule V of Drugs and Cosmetics Rules)
5. Nicotinamide
6. Pyridoxine
7. Riboflavin
8. Thiamine
9. Vitamin A
10. Vitamin D (Ergocalciferol)

Page 93 of 123
---
|S.No|Alphabetical List of Medicines in NLEM 2011|
|---|---|
|1.|25% Dextrose|
|2.|5-Amino salicylic Acid|
|3.|5-Fluorouracil|
|4.|Acetazolamide|
|5.|Acetyl Salicylic Acid|
|6.|Acriflavin+Glycerin|
|7.|Actinomycin D|
|8.|Activated Charcoal|
|9.|Acyclovir|
|10.|Adenosine|
|11.|Adrenaline Bitartrate|
|12.|Albendazole|
|13.|Albumin|
|14.|Allopurinol|
|15.|Alpha Interferon|
|16.|Alprazolam|
|17.|Aluminium Hydroxide + Magnesium Hydroxide|
|18.|Amikacin|
|19.|Amiodarone|
|20.|Amitriptyline|
|21.|Amlodipine|
|22.|Amoxicillin|
|23.|Amoxicillin+Clavulinic acid|
|24.|Amphotericin B|

Page 94 of 123
---
|25.|Ampicillin|
|---|---|
|26.|Anti-D immunoglobin (human)|
|27.|Antitetanus Human immunoglobin|
|28.|Artesunate (To be used only in combination with Sulfadoxine + Pyrimethamine)|
|29.|Ascorbic Acid|
|30.|Atenolol|
|31.|Atorvastatin|
|32.|Atracurium besylate|
|33.|Atropine Sulphate|
|34.|Azathioprine|
|35.|Azithromycin|
|36.|B.C.G Vaccine|
|37.|Barium Sulphate|
|38.|Beclomethasone Dipropionate|
|39.|Benzathine Benzylpenicillin|
|40.|Benzoin Compound|
|41.|Benzyl benzoate|
|42.|Betamethasone|
|43.|Betamethasone Dipropionate|
|44.|Betaxolol Hydrochloride|
|45.|Bisacodyl|
|46.|Bleaching Powder|
|47.|Bleomycin|
|48.|Bromocriptine Mesylate|
|49.|Bupivacaine Hydrochloride|

Page 95 of 123
---
|50.|Busulphan|
|---|---|
|51.|Calamine|
|52.|Calcium carbonate|
|53.|Calcium gluconate|
|54.|Calcium Ipodate|
|55.|Carbamazepine|
|56.|Carbimazole|
|57.|Carboplatin|
|58.|Cefixime|
|59.|Cefotaxime|
|60.|Ceftazidime|
|61.|Ceftriaxone|
|62.|Cephalexin|
|63.|Cetrimide|
|64.|Cetrizine|
|65.|Chlorambucil|
|66.|Chloramphenicol|
|67.|Chlorhexidine|
|68.|Chloroquine phosphate|
|69.|Chlorpheniramine Maleate|
|70.|Chlorpromazine hydrochloride|
|71.|Ciprofloxacin Hydrochloride|
|72.|Cisplatin|
|73.|Clindamycin|
|74.|Clofazimine|
|75.|Clomiphene citrate|
---
|76.|Clopidogrel|
|---|---|
|77.|Clotrimazole|
|78.|Cloxacillin|
|79.|Coal Tar|
|80.|Codeine phosphate|
|81.|Colchicine|
|82.|Condoms|
|83.|Co-Trimoxazole (Trimethoprim + Sulphamethoxazole)|
|84.|Cryoprecipitate|
|85.|Cyanocobalamin|
|86.|Cyclophosphamide|
|87.|Cyclosporine|
|88.|Cytosine arabinoside|
|89.|D.P.T Vaccine|
|90.|Dacarbazine|
|91.|Danazol|
|92.|Dapsone|
|93.|Daunorubicin|
|94.|Desferrioxamine mesylate|
|95.|Dexamethasone|
|96.|Dexchlorpheniramine Maleate|
|97.|Dextran-40|
|98.|Dextran-70|
|99.|Dextromethorphan|
|100.|Diazepam|
|101.|Diclofenac|
---
# Page 98 of 123

102. Dicyclomine Hydrochloride
103. Didanosine
104. Diepylcarbamazine citrate
105. Digoxin
106. Dihydroergotamine
107. Diloxanide Furoate
108. Diltiazem
109. Dimercaprol
110. Diphperia Antitoxin
111. Dipranol
112. Dobutamine
113. Domperidone
114. Dopamine Hydrochloride
115. Doxorubicin
116. Doxycycline
117. Efavirenz
118. EMLA cream
119. Enalapril Maleate
120. Enoxaparin
121. Erypromycin Estolate
122. Esmolol
123. Epambutol
124. Eper
125. Epinylestradiol
126. Epinylestradiol + Levonorgesterol
127. Epinylestradiol + Norepisterone
---
128. Epyl Alcohol 70%
129. Etoposide
130. Factor IX Complex (Coagulation Factors II,VII, IX, X)
131. Factor VIII Concentrate
132. Famotidine
133. Fentanyl
134. Ferrous Salt
135. Filgrastim
136. Fluconazole
137. Flumazenil
138. Fluorescein
139. Fluoxetine hydrochloride
140. Flutamide
141. Folic Acid
142. Folinic Acid
143. Formaldehyde IP
144. Framycetin Sulphate
145. Fresh frozen plasma
146. Furosemide
147. Gemcitabine hydrochloride
148. Gentamicin
149. Gentian Violet
150. Glibenclamide
151. Glucagon
152. Glucose
153. Glucose wip sodium chloride
---
|154.|Glutaraldehyde|
|---|---|
|155.|Glycerin|
|156.|Glyceryl Trinitrate|
|157.|Griseofulvin|
|158.|Haloperidol|
|159.|Halothane with vaporizer|
|160.|Heparin Sodium|
|161.|Hepatitis B Vaccine|
|162.|Homatropine|
|163.|Hormone Releasing IUD|
|164.|Hydrochlorothiazide|
|165.|Hydrocortisone sodium succinate|
|166.|Hydrogen Peroxide|
|167.|Hydroxychloroquine phosphate|
|168.|Hydroxyethyl Starch (Hetastarch)|
|169.|Hyoscine Butyl Bromide|
|170.|Ibuprofen|
|171.|Ifosfamide|
|172.|Imatinib|
|173.|Imipramine|
|174.|Indinavir|
|175.|Insulin Injection(Soluble)|
|176.|Intermediate Acting(Lente/NPH Insulin)|
|177.|Intraperitoneal Dialysis Solution|
|178.|Iodine|
|179.|Iopanoic Acid|
---
180. Ipratropium bromide
181. Iron Dextran
182. Isoflurane
183. Isoniazid
184. Isosorbide 5 Mononitrate/Dinitrate
185. Ispaghula
186. IUD containing Copper
187. Ketamine Hydrochloride
188. L- Asparaginase
189. Lamivudine
190. Lamivudine + Nevirapine + Stavudine
191. Lamivudine + Zidovudine
192. Leflunomide
193. Levodopa+ Carbidopa
194. Levopyroxine
195. Lignocaine
196. Lignocaine Hydrochloride
197. Lignocaine Hydrochloride + Adrenaline
198. Lipium Carbonate
199. Lorazepam
200. Losartan Potassium
201. Magnesium sulphate
202. Mannitol
203. Measles Vaccine
204. Medroxy Progesterone Acetate
205. Mefloquine

Page 101 of 123
---
206. Meglumine Iopalamate
207. Meglumine Iotroxate
208. Melphalan
209. Mercaptopurine
210. Mesna
211. Metformin
212. Mepotrexate
213. Mepyl Cellulose
214. Mepyl Ergometrine
215. Mepyldopa
216. MepylPrednisolone
217. Mepylrosanilinium Chloride (Gentian Violet)
218. Mepylpioninium chloride (Mepylene blue)
219. Metoclopramide
220. Metoprolol
221. Metronidazole
222. Miconazole
223. Midazolam
224. Mifepristone
225. Misoprostol
226. Mitomycin-C
227. Morphine Sulphate
228. Multivitamins (As per Schedule V of Drugs and Cosmetics Rules)
229. N/2 Saline
230. N/5 Saline
---
231. N-acetylcysteine
232. Naloxone
233. Nelfinavir
234. Neomycin + Bacitracin
235. Neostigmine
236. Nevirapine
237. Nicotinamide
238. Nifedipine
239. Nitrofurantoin
240. Nitrous Oxide
241. Norepisterone
242. Normal Saline
243. Nystatin
244. Ofloxacin
245. Olanzapine
246. Omeprazole
247. Ondansetron
248. Oral Poliomyelitis vaccine (LA)
249. Oral Rehydration Salts
250. Oxaliplatin
251. Oxygen
252. Oxytocin
253. Paclitaxel
254. Pantoprazole
255. Paracetamol
256. Penicillamine

Page 103 of 123
---
257. Pentamidine Isopionate
258. Permeprin
259. Pheniramine Maleate
260. Phenobarbitone
261. Phenylephrine
262. Phenytoin Sodium
263. Phytomenadione
264. Pilocarpine
265. Piperazine
266. Platelet Rich Plasma
267. Polygeline
268. Polyvalent Antisnake Venom
269. Potassium Chloride
270. Potassium Permanganate
271. Povidone Iodine
272. Pralidoxime Chloride(2-PAM)
273. Praziquantel
274. Prednisolone
275. Prednisolone Acetate
276. Prednisolone Sodium Phosphate
277. Premix Insulin 30:70 injection
278. Primaquine
279. Procainamide Hydrochloride
280. Procarbazine
281. Promepazine
282. Propofol

Page 104 of 123
---
283. Propranolol hydrochloride
284. Propyliodone
285. Protamine Sulphate
286. Pyrazinamide
287. Pyridostigmine
288. Pyridoxine
289. Pyrimepamine
290. Quinine sulphate
291. Rabies immunoglobin
292. Rabies Vaccine
293. Raloxifene
294. Ranitidine
295. Riboflavin
296. Rifampicin
297. Ringer Lactate
298. Ritonavir
299. Salbutamol sulphate
300. Salicylic Acid
301. Saquinavir
302. Sevoflurane
303. Silver Sulphadiazine
304. Sodium Bicarbonate
305. Sodium Iopalamate
306. Sodium Meglumine Diatrizoate
307. Sodium Nitrite
308. Sodium Nitroprusside
---
309. Sodium Stibogluconate
310. Sodium Thiosulphate
311. Sodium Valproate
312. Specific antisnake venom
313. Spironolactone
314. Stavudine
315. Stavudine+Lamivudine
316. Streptokinase
317. Streptomycin Sulphate
318. Succinyl choline chloride
319. Sulfadoxine + Pyrimepamine
320. Sulfasalazine
321. Sulphacetamide Sodium
322. Sulphadiazine
323. Tamoxifen Citrate
324. Terbutaline Sulphate
325. Testosterone
326. Tetanus Toxoid
327. Tetracaine Hydrochloride
328. Thiamine
329. Thiopentone Sodium
330. Timolol Maleate
331. Tramadol
332. Trihexyphenidyl Hydrochloride
333. Tropicamide
334. Tuberculin, Purified Protein derivative
---
335. Urokinase
336. Vancomycin Hydrochloride
337. Vecuronium
338. Verapamil
339. Vinblastine sulphate
340. Vincristine
341. Vitamin A
342. Vitamin D (Ergocalciferol)
343. Warfarin sodium
344. Water for Injection
345. Zidovudine
346. Zidovudine+ Lamivudine+ Nevirapine
347. Zinc Oxide
348. Zinc Sulfate

Page 107 of 123
---
|S.No|List of Medicines in NLEM 2011 – Category Wise|Category|
|---|---|---|
|1.|25% Dextrose|P,S,T|
|2.|Acetyl Salicylic Acid|P,S,T|
|3.|Acriflavin+Glycerin|P,S,T|
|4.|Activated Charcoal|P,S,T|
|5.|Adrenaline Bitartrate|P,S,T|
|6.|Albendazole|P,S,T|
|7.|Alprazolam|P,S,T|
|8.|Aluminium Hydroxide + Magnesium Hydroxide|P,S,T|
|9.|Amitriptyline|P,S,T|
|10.|Amlodipine|P,S,T|
|11.|Ampicillin|P,S,T|
|12.|Antitetanus Human immunoglobin|P,S,T|
|13.|Artesunate (To be used only in combination with Sulfadoxine + Pyrimethamine)|P,S,T|
|14.|Ascorbic Acid|P,S,T|
|15.|Atenolol|P,S,T|
|16.|Atorvastatin|P,S,T|
|17.|Atropine Sulphate|P,S,T|
|18.|B.C.G Vaccine|P,S,T|
|19.|Beclomethasone Dipropionate|P,S,T|
|20.|Benzathine Benzylpenicillin|P,S,T|

Page 108 of 123
---
|S.No|List of Medicines in NLEM 2011 – Category Wise|Category|
|---|---|---|
|21.|Benzoin Compound|P,S,T|
|22.|Benzyl benzoate|P,S,T|
|23.|Betamethasone|P,S,T|
|24.|Betamethasone Dipropionate|P,S,T|
|25.|Bisacodyl|P,S,T|
|26.|Bleaching Powder|P,S,T|
|27.|Calamine|P,S,T|
|28.|Calcium carbonate|P,S,T|
|29.|Calcium gluconate|P,S,T|
|30.|Carbamazepine|P,S,T|
|31.|Carbimazole|P,S,T|
|32.|Cephalexin|P,S,T|
|33.|Cetrimide|P,S,T|
|34.|Cetrizine|P,S,T|
|35.|Chloramphenicol|P,S,T|
|36.|Chlorhexidine|P,S,T|
|37.|Chloroquine phosphate|P,S,T|
|38.|Chlorpheniramine Maleate|P,S,T|
|39.|Chlorpromazine hydrochloride|P,S,T|
|40.|Ciprofloxacin Hydrochloride|P,S,T|
|41.|Clofazimine|P,S,T|
|42.|Clotrimazole|P,S,T|
|43.|Cloxacillin|P,S,T|

Page 109 of 123
---
|S.No|List of Medicines in NLEM 2011 – Category Wise|Category|
|---|---|---|
|44.|Coal Tar|P,S,T|
|45.|Condoms|P,S,T|
|46.|Co-Trimoxazole (Trimethoprim + Sulphamethoxazole)|P,S,T|
|47.|Cyanocobalamin|P,S,T|
|48.|D.P.T Vaccine|P,S,T|
|49.|Dapsone|P,S,T|
|50.|Dexamethasone|P,S,T|
|51.|Dexchlorpheniramine Maleate|P,S,T|
|52.|Dextran-40|P,S,T|
|53.|Dextran-70|P,S,T|
|54.|Dextromethorphan|P,S,T|
|55.|Diazepam|P,S,T|
|56.|Dicyclomine Hydrochloride|P,S,T|
|57.|Diethylcarbamazine citrate|P,S,T|
|58.|Diloxanide Furoate|P,S,T|
|59.|Domperidone|P,S,T|
|60.|Doxycycline|P,S,T|
|61.|Enalapril Maleate|P,S,T|
|62.|Erythromycin Estolate|P,S,T|
|63.|Ethambutol|P,S,T|
|64.|Ethinylestradiol|P,S,T|
|65.|Ethinylestradiol + Levonorgesterol|P,S,T|
|66.|Ethinylestradiol + Norethisterone|P,S,T|

Page 110 of 123
---
|S.No|List of Medicines in NLEM 2011 – Category Wise|Category|
|---|---|---|
|67.|Ethyl Alcohol 70%|P,S,T|
|68.|Famotidine|P,S,T|
|69.|Ferrous Salt|P,S,T|
|70.|Fluoxetine hydrochloride|P,S,T|
|71.|Folic Acid|P,S,T|
|72.|Formaldehyde IP|P,S,T|
|73.|Framycetin Sulphate|P,S,T|
|74.|Furosemide|P,S,T|
|75.|Gentamicin|P,S,T|
|76.|Gentian Violet|P,S,T|
|77.|Glibenclamide|P,S,T|
|78.|Glucose|P,S,T|
|79.|Glucose with sodium chloride|P,S,T|
|80.|Glycerin|P,S,T|
|81.|Glyceryl Trinitrate|P,S,T|
|82.|Griseofulvin|P,S,T|
|83.|Hepatitis B Vaccine|P,S,T|
|84.|Homatropine|P,S,T|
|85.|Hydrochlorothiazide|P,S,T|
|86.|Hydrocortisone sodium succinate|P,S,T|
|87.|Hydrogen Peroxide|P,S,T|
|88.|Hyoscine Butyl Bromide|P,S,T|
|89.|Ibuprofen|P,S,T|

Page 111 of 123
---
|S.No|List of Medicines in NLEM 2011 – Category Wise|Category|
|---|---|---|
|90.|Imipramine|P,S,T|
|91.|Insulin Injection(Soluble)|P,S,T|
|92.|Intermediate Acting(Lente/NPH Insulin)|P,S,T|
|93.|Ipratropium bromide|P,S,T|
|94.|Isoniazid|P,S,T|
|95.|Isosorbide 5 Mononitrate/Dinitrate|P,S,T|
|96.|Ispaghula|P,S,T|
|97.|IUD containing Copper|P,S,T|
|98.|Ketamine Hydrochloride|P,S,T|
|99.|Levodopa+ Carbidopa|P,S,T|
|100.|Levothyroxine|P,S,T|
|101.|Lignocaine Hydrochloride|P,S,T|
|102.|Lignocaine Hydrochloride + Adrenaline|P,S,T|
|103.|Mannitol|P,S,T|
|104.|Measles Vaccine|P,S,T|
|105.|Medroxy Progesterone Acetate|P,S,T|
|106.|Metformin|P,S,T|
|107.|Methyl Ergometrine|P,S,T|
|108.|Methyldopa|P,S,T|
|109.|Methylrosanilinium Chloride (Gentian Violet)|P,S,T|
|110.|Metoclopramide|P,S,T|
|111.|Metoprolol|P,S,T|
|112.|Metronidazole|P,S,T|

Page 112 of 123
---
|S.No|List of Medicines in NLEM 2011 – Category Wise|Category|
|---|---|---|
|113.|Miconazole|P,S,T|
|114.|Midazolam|P,S,T|
|115.|Multivitamins (As per Schedule V of Drugs and Cosmetics Rules)|P,S,T|
|116.|N-acetylcysteine|P,S,T|
|117.|Naloxone|P,S,T|
|118.|Neomycin + Bacitracin|P,S,T|
|119.|Nicotinamide|P,S,T|
|120.|Nitrofurantoin|P,S,T|
|121.|Nitrous Oxide|P,S,T|
|122.|Norethisterone|P,S,T|
|123.|Normal Saline|P,S,T|
|124.|Nystatin|P,S,T|
|125.|Omeprazole|P,S,T|
|126.|Oral Poliomyelitis vaccine (LA)|P,S,T|
|127.|Oral Rehydration Salts|P,S,T|
|128.|Oxygen|P,S,T|
|129.|Paracetamol|P,S,T|
|130.|Pheniramine Maleate|P,S,T|
|131.|Phenobarbitone|P,S,T|
|132.|Phenylephrine|P,S,T|
|133.|Phenytoin Sodium|P,S,T|
|134.|Phytomenadione|P,S,T|
|135.|Piperazine|P,S,T|

Page 113 of 123
---
|S.No|List of Medicines in NLEM 2011 – Category Wise|Category|
|---|---|---|
|136.|Polyvalent Antisnake Venom|P,S,T|
|137.|Potassium Chloride|P,S,T|
|138.|Potassium Permanganate|P,S,T|
|139.|Povidone Iodine|P,S,T|
|140.|Pralidoxime Chloride(2-PAM)|P,S,T|
|141.|Prednisolone|P,S,T|
|142.|Prednisolone Acetate|P,S,T|
|143.|Prednisolone Sodium Phosphate|P,S,T|
|144.|Premix Insulin 30:70 injection|P,S,T|
|145.|Primaquine|P,S,T|
|146.|Promethazine|P,S,T|
|147.|Propofol|P,S,T|
|148.|Propranolol hydrochloride|P,S,T|
|149.|Pyrazinamide|P,S,T|
|150.|Pyridoxine|P,S,T|
|151.|Pyrimethamine|P,S,T|
|152.|Quinine sulphate|P,S,T|
|153.|Rabies immunoglobin|P,S,T|
|154.|Rabies Vaccine|P,S,T|
|155.|Ranitidine|P,S,T|
|156.|Riboflavin|P,S,T|
|157.|Rifampicin|P,S,T|
|158.|Ringer Lactate|P,S,T|

Page 114 of 123
---
|S.No|List of Medicines in NLEM 2011 – Category Wise|Category|
|---|---|---|
|159.|Salbutamol sulphate|P,S,T|
|160.|Salicylic Acid|P,S,T|
|161.|Silver Sulphadiazine|P,S,T|
|162.|Sodium Bicarbonate|P,S,T|
|163.|Sodium Valproate|P,S,T|
|164.|Specific antisnake venom|P,S,T|
|165.|Spironolactone|P,S,T|
|166.|Streptomycin Sulphate|P,S,T|
|167.|Sulfadoxine + Pyrimethamine|P,S,T|
|168.|Sulphacetamide Sodium|P,S,T|
|169.|Testosterone|P,S,T|
|170.|Tetanus Toxoid|P,S,T|
|171.|Tetracaine Hydrochloride|P,S,T|
|172.|Thiamine|P,S,T|
|173.|Timolol Maleate|P,S,T|
|174.|Trihexyphenidyl Hydrochloride|P,S,T|
|175.|Tuberculin, Purified Protein derivative|P,S,T|
|176.|Vecuronium|P,S,T|
|177.|Vitamin A|P,S,T|
|178.|Vitamin D (Ergocalciferol)|P,S,T|
|179.|Water for Injection|P,S,T|
|180.|Zinc Oxide|P,S,T|
|181.|Zinc Sulfate|P,S,T|

Page 115 of 123
---
|S.No|List of Medicines in NLEM 2011 – Category Wise|Category|
|---|---|---|
|1.|5-Amino salicylic Acid|S,T|
|2.|Acetazolamide|S,T|
|3.|Acyclovir|S, T|
|4.|Adenosine|S,T|
|5.|Albumin|S, T|
|6.|Allopurinol|S, T|
|7.|Amikacin|S, T|
|8.|Amiodarone|S, T|
|9.|Amoxicillin|S,T|
|10.|Amphotericin B|S, T|
|11.|Anti-D immunoglobin (human)|S, T|
|12.|Atracurium besylate|S, T|
|13.|Azathioprine|S, T|
|14.|Azithromycin|S,T|
|15.|Barium Sulphate|S, T|
|16.|Bromocriptine Mesylate|S, T|
|17.|Bupivacaine Hydrochloride|S, T|
|18.|Calcium Ipodate|S, T|
|19.|Cefotaxime|S, T|
|20.|Ceftazidime|S, T|
|21.|Ceftriaxone|S, T|
---
|S.No|List of Medicines in NLEM 2011 – Category Wise|Category|
|---|---|---|
|22.|Clindamycin|S, T|
|23.|Codeine phosphate|S, T|
|24.|Colchicine|S, T|
|25.|Cryoprecipitate|S, T|
|26.|Desferrioxamine mesylate|S, T|
|27.|Didanosine|S, T|
|28.|Digoxin|S, T|
|29.|Dihydroergotamine|S, T|
|30.|Diltiazem|S, T|
|31.|Dimercaprol|S, T|
|32.|Diphtheria Antitoxin|S, T|
|33.|Dobutamine|S, T|
|34.|Dopamine Hydrochloride|S, T|
|35.|Efavirenz|S, T|
|36.|Ether|S, T|
|37.|Factor IX Complex (Coagulation Factors II,VII, IX, X)|S, T|
|38.|Factor VIII Concentrate|S, T|
|39.|Fentanyl|S, T|
|40.|Fluconazole|S, T|
|41.|Fluorescein|S, T|
|42.|Glutaraldehyde|S, T|
|43.|Haloperidol|S, T|
|44.|Halothane with vaporizer|S, T|

Page 117 of 123
---
|S.No|List of Medicines in NLEM 2011 – Category Wise|Category|
|---|---|---|
|45.|Heparin Sodium|S, T|
|46.|Hydroxychloroquine phosphate|S, T|
|47.|Hydroxyethyl Starch (Hetastarch)|S, T|
|48.|Indinavir|S, T|
|49.|Iodine|S, T|
|50.|Iopanoic Acid|S, T|
|51.|Iron Dextran|S, T|
|52.|Isoflurane|S, T|
|53.|Lamivudine|S, T|
|54.|Lamivudine + Nevirapine + Stavudine|S, T|
|55.|Lamivudine + Zidovudine|S, T|
|56.|Leflunomide|S, T|
|57.|Lignocaine|S, T|
|58.|Losartan Potassium|S, T|
|59.|Magnesium sulphate|S, T|
|60.|Mefloquine|S, T|
|61.|Meglumine Iothalamate|S, T|
|62.|Meglumine Iotroxate|S, T|
|63.|Methotrexate|S, T|
|64.|MethylPrednisolone|S, T|
|65.|Methylthioninium chloride (Methylene blue)|S, T|
|66.|Morphine Sulphate|S, T|
|67.|N/2 Saline|S, T|

Page 118 of 123
---
|S.No|List of Medicines in NLEM 2011 – Category Wise|Category|
|---|---|---|
|68.|N/5 Saline|S, T|
|69.|Nelfinavir|S, T|
|70.|Neostigmine|S, T|
|71.|Nevirapine|S, T|
|72.|Nifedipine|S, T|
|73.|Ofloxacin|S, T|
|74.|Ondansetron|S, T|
|75.|Oxytocin|S, T|
|76.|Penicillamine|S, T|
|77.|Pentamidine Isothionate|S, T|
|78.|Permethrin|S, T|
|79.|Pilocarpine|S, T|
|80.|Platelet Rich Plasma|S, T|
|81.|Polygeline|S, T|
|82.|Praziquantel|S, T|
|83.|Propyliodone|S, T|
|84.|Protamine Sulphate|S, T|
|85.|Pyridostigmine|S, T|
|86.|Ritonavir|S, T|
|87.|Saquinavir|S, T|
|88.|Sodium Iothalamate|S, T|
|89.|Sodium Meglumine Diatrizoate|S, T|
|90.|Sodium Nitrite|S, T|

Page 119 of 123
---
|S.No|List of Medicines in NLEM 2011 – Category Wise|Category|
|---|---|---|
|91.|Sodium Stibogluconate|S, T|
|92.|Sodium Thiosulphate|S, T|
|93.|Stavudine|S, T|
|94.|Stavudine+Lamivudine|S, T|
|95.|Streptokinase|S, T|
|96.|Succinyl choline chloride|S, T|
|97.|Sulfasalazine|S, T|
|98.|Sulphadiazine|S, T|
|99.|Terbutaline Sulphate|S, T|
|100.|Thiopentone Sodium|S, T|
|101.|Tramadol|S, T|
|102.|Tropicamide|S, T|
|103.|Verapamil|S, T|
|104.|Warfarin sodium|S, T|
|105.|Zidovudine|S, T|
|106.|Zidovudine+ Lamivudine+ Nevirapine|S, T|

Total Medicines under Category S,T - 106

|S.No|List of Medicines in NLEM 2011 – Category Wise|Category|
|---|---|---|
|1.|5-Fluorouracil|T|
|2.|Actinomycin D|T|
|3.|Alpha Interferon|T|
|4.|Amoxicillin+Clavulinic acid|T|
|5.|Betaxolol Hydrochloride|T|
---
|S.No|List of Medicines in NLEM 2011 – Category Wise|Category|
|---|---|---|
|6.|Bleomycin|T|
|7.|Busulphan|T|
|8.|Carboplatin|T|
|9.|Cefixime|T|
|10.|Chlorambucil|T|
|11.|Cisplatin|T|
|12.|Clomiphene citrate|T|
|13.|Clopidogrel|T|
|14.|Cyclophosphamide|T|
|15.|Cyclosporine|T|
|16.|Cytosine arabinoside|T|
|17.|Dacarbazine|T|
|18.|Danazol|T|
|19.|Daunorubicin|T|
|20.|Diclofenac|T|
|21.|Dithranol|T|
|22.|Doxorubicin|T|
|23.|EMLA cream|T|
|24.|Enoxaparin|T|
|25.|Esmolol|T|
|26.|Etoposide|T|
|27.|Filgrastim|T|
|28.|Flumazenil|T|

Page 121 of 123
---
|S.No|List of Medicines in NLEM 2011 – Category Wise|Category|
|---|---|---|
|29.|Flutamide|T|
|30.|Folinic Acid|T|
|31.|Fresh frozen plasma|T|
|32.|Gemcitabine hydrochloride|T|
|33.|Glucagon|T|
|34.|Hormone Releasing IUD|T|
|35.|Ifosfamide|T|
|36.|Imatinib|T|
|37.|Intraperitoneal Dialysis Solution|T|
|38.|L- Asparaginase|T|
|39.|Lithium Carbonate|T|
|40.|Lorazepam|T|
|41.|Melphalan|T|
|42.|Mercaptopurine|T|
|43.|Mesna|T|
|44.|Methyl Cellulose|T|
|45.|Mifepristone|T|
|46.|Misoprostol|T|
|47.|Mitomycin-C|T|
|48.|Olanzapine|T|
|49.|Oxaliplatin|T|
|50.|Paclitaxel|T|
|51.|Pantoprazole|T|
---
|S.No|List of Medicines in NLEM 2011 – Category Wise|Category|
|---|---|---|
|52.|Procainamide Hydrochloride|T|
|53.|Procarbazine|T|
|54.|Raloxifene|T|
|55.|Sevoflurane|T|
|56.|Sodium Nitroprusside|T|
|57.|Tamoxifen Citrate|T|
|58.|Urokinase|T|
|59.|Vancomycin Hydrochloride|T|
|60.|Vinblastine sulphate|T|
|61.|Vincristine|T|

|Total Medicines| |
|---|---|
|Category P,S,T|181|
|Category S,T|106|
|Category T|61|
|Total|348

In [10]:
document_path = Path("data/parsed_document.md")
with document_path.open("a") as f:
    f.write(parsed_doc[0].text)

In [11]:

loader = UnstructuredMarkdownLoader(document_path)
loaded_documents = loader.load()



[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [12]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=2048, chunk_overlap=128)
docs = text_splitter.split_documents(loaded_documents)
len(docs)

49

In [13]:
print(docs[0].page_content)

National List of Essential Medicines of India 2011
Page 1 of 123

TABLE OF CONTENTS

PREAMBLE ...................................................................................................................................... 4

EXECUTIVE SUMMARY ................................................................................................................... 6

Salient features of NLEM 2011: ............................................................................................................ …..11

The meetings/deliberations of core committee/ National consultation meetings held for
preparing the National List of Essential Medicines 2011 ............................................................. 12

PROCESS ADOPTED FOR REVISION OF NLEM .............................................................................. 13

POTENTIAL USES OF NLEM ........................................................................................................... 15

EXPERTS WHO PARTICIPA

In [14]:
embeddings = FastEmbedEmbeddings(model_name="BAAI/bge-base-en-v1.5")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/740 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.24k [00:00<?, ?B/s]

model_optimized.onnx:   0%|          | 0.00/218M [00:00<?, ?B/s]

In [15]:
qdrant = Qdrant.from_documents(
    docs,
    embeddings,
    # location=":memory:",
    path="./db",
    collection_name="document_embeddings",
)


In [16]:

%%time
query = "What is Lithium Carbonate used for ?"
similar_docs = qdrant.similarity_search_with_score(query)

CPU times: user 332 ms, sys: 7.04 ms, total: 339 ms
Wall time: 342 ms


In [17]:

for doc, score in similar_docs:
    print(f"text: {doc.page_content[:256]}\n")
    print(f"score: {score}")
    print("-" * 80)
    print()

text: 5. Dimercaprol 6. Flumazenil 7. Methylthioninium chloride (Methylene blue) 8. N-acetylcysteine 9. Naloxone 10. Penicillamine 11. Pralidoxime Chloride(2-PAM) 12. Sodium Nitrite 13. Sodium Thiosulphate 14. Specific antisnake venom

SECTION: 5 – Anticonvulsan

score: 0.5375497311668944
--------------------------------------------------------------------------------

text: Page 121 of 123

S.No List of Medicines in NLEM 2011 – Category Wise Category 29. Flutamide T 30. Folinic Acid T 31. Fresh frozen plasma T 32. Gemcitabine hydrochloride T 33. Glucagon T 34. Hormone Releasing IUD T 35. Ifosfamide T 36. Imatinib T 37. Intrap

score: 0.5341344330801845
--------------------------------------------------------------------------------

text: Meglumine Iotroxate

Melphalan

Mercaptopurine

Mesna

Metformin

Mepotrexate

Mepyl Cellulose

Mepyl Ergometrine

Mepyldopa

MepylPrednisolone

Mepylrosanilinium Chloride (Gentian Violet)

Mepylpioninium chloride (Mepylene blue)

Metoclopramide

Met

In [18]:
%%time
retriever = qdrant.as_retriever(search_kwargs={"k": 5})
retrieved_docs = retriever.invoke(query)

CPU times: user 359 ms, sys: 4.04 ms, total: 363 ms
Wall time: 359 ms


In [19]:
for doc in retrieved_docs:
    print(f"id: {doc.metadata['_id']}\n")
    print(f"text: {doc.page_content[:256]}\n")
    print("-" * 80)
    print()


id: 65f13061c40243b38a943843a9d350c7

text: 5. Dimercaprol 6. Flumazenil 7. Methylthioninium chloride (Methylene blue) 8. N-acetylcysteine 9. Naloxone 10. Penicillamine 11. Pralidoxime Chloride(2-PAM) 12. Sodium Nitrite 13. Sodium Thiosulphate 14. Specific antisnake venom

SECTION: 5 – Anticonvulsan

--------------------------------------------------------------------------------

id: f9b0939c68e94ff4b6df9abf95a5de36

text: Page 121 of 123

S.No List of Medicines in NLEM 2011 – Category Wise Category 29. Flutamide T 30. Folinic Acid T 31. Fresh frozen plasma T 32. Gemcitabine hydrochloride T 33. Glucagon T 34. Hormone Releasing IUD T 35. Ifosfamide T 36. Imatinib T 37. Intrap

--------------------------------------------------------------------------------

id: ab80c97e18ad472c838ede253d3147bf

text: Meglumine Iotroxate

Melphalan

Mercaptopurine

Mesna

Metformin

Mepotrexate

Mepyl Cellulose

Mepyl Ergometrine

Mepyldopa

MepylPrednisolone

Mepylrosanilinium Chloride (Gentian Violet)

In [20]:

compressor = FlashrankRerank(model="ms-marco-MiniLM-L-12-v2")
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=retriever
)


ms-marco-MiniLM-L-12-v2.zip: 100%|██████████| 21.6M/21.6M [00:00<00:00, 113MiB/s] 


In [21]:
%%time
reranked_docs = compression_retriever.invoke(query)
len(reranked_docs)

Running pairwise ranking..
CPU times: user 3.78 s, sys: 276 ms, total: 4.06 s
Wall time: 4.16 s


3

In [22]:
for doc in reranked_docs:
    print(f"id: {doc.metadata['_id']}\n")
    print(f"text: {doc.page_content[:256]}\n")
    print(f"score: {doc.metadata['relevance_score']}")
    print("-" * 80)
    print()

id: eed066476e0b48359d4e7d0fb52f44c0

text: SECTION: 24 – Psychotherapeutic Medicines

1. Alprazolam 2. Amitriptyline 3. Chlorpromazine hydrochloride

Page 91 of 123

Alphabetical List of Medicines – Therapeutic area wise

4. Diazepam 5. Fluoxetine hydrochloride 6. Haloperidol 7. Imipramine 8. Lithi

score: 0.036652203649282455
--------------------------------------------------------------------------------

id: 79903f687dc24d169ed83d08eee7a209

text: Page 75 of 123

Medicines Category Route of Administration/Dosage Form Strengths Ascorbic Acid P,S,T Tablets 100 mg, 500 mg Calcium carbonate P,S,T Tablets 250 mg, 500 mg Multivitamins (As per Schedule V of Drugs and Cosmetics Rules) P,S,T Tablets Nicotina

score: 0.00843687541782856
--------------------------------------------------------------------------------

id: f9b0939c68e94ff4b6df9abf95a5de36

text: Page 121 of 123

S.No List of Medicines in NLEM 2011 – Category Wise Category 29. Flutamide T 30. Folinic Acid T 31. Fresh frozen pla

In [23]:
llm = ChatGroq(temperature=0, model_name="llama3-70b-8192")

In [24]:
prompt_template = """
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer. However, make sure you are also as enagaing as possible.
Make you sure you intract with the user in a human like manner. and also engage with the user in a conversational and empathetic manner

Context: {context}
Question: {question}

Answer the question and provide additional helpful information,
based on the pieces of information, if applicable. Be succinct.

Responses should be properly formatted to be easily read.
"""

prompt = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

In [25]:

qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=compression_retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt, "verbose": True},
)

In [26]:
%%time
response = qa.invoke("What is Lithium Carbonate used for ?")

Running pairwise ranking..


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer. However, make sure you are also as enagaing as possible.
Make you sure you intract with the user in a human like manner. and also engage with the user in a conversational and empathetic manner

Context: SECTION: 24 – Psychotherapeutic Medicines

1. Alprazolam 2. Amitriptyline 3. Chlorpromazine hydrochloride

Page 91 of 123

Alphabetical List of Medicines – Therapeutic area wise

4. Diazepam 5. Fluoxetine hydrochloride 6. Haloperidol 7. Imipramine 8. Lithium Carbonate 9. Olanzapine 10. Sodium Valproate

SECTION: 25 – Medicines acting on the respiratory tract

1. Beclomethasone Dipropionate 2. Codeine phosphate 3. Dextromethorphan 4. Hydrocortisone sodium succinate 5. Ipratropium bromide 6. Salbu

In [27]:
print_response(response)

**Answer:** Unfortunately, the provided information does not explicitly state what Lithium Carbonate
is used for. However, I can tell you that it is a psychotherapeutic medicine, as it is listed under
SECTION: 24 – Psychotherapeutic Medicines.

**Additional Information:** Lithium Carbonate is commonly used to treat bipolar disorder, a mental
health condition characterized by periods of mania or hypomania, often alternating with episodes of
depression. It helps to stabilize mood and reduce symptoms of mania.

If you have any further questions or concerns, feel free to ask!


In [28]:
%%time
response = qa.invoke("What is Lithium Carbonate used for and what dosages can i take for a child in their teens ")

Running pairwise ranking..


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer. However, make sure you are also as enagaing as possible.
Make you sure you intract with the user in a human like manner. and also engage with the user in a conversational and empathetic manner

Context: Medicines Category Route of Administration/Dosage Form Strengths Atropine Sulphate P, S, T Drops/Ointment 1% Homatropine P, S, T Drops 2% Phenylephrine P, S, T Drops 5%

Medicines Category Route of Administration/Dosage Form Strengths Methyl Cellulose T Injection 2%

Medicines Category Route of Administration/Dosage Form Strengths Methyl Ergometrine P, S, T Injection 0.2mg/ml Mifepristone T Tablets 200mg Oxytocin S, T Injection 5 IU/ml, 10 IU/ml --- Added medicines --- --- --- --- Misoprostol T 

In [29]:
print_response(response)

**Lithium Carbonate: Uses and Dosages for Teenagers**

According to the provided information, Lithium Carbonate is used to treat **Bipolar disorders**.

The available dosage form and strength for Lithium Carbonate are:

* Tablets: 300 mg

Unfortunately, the provided information does not specify the dosage for children in their teens.
It's essential to consult a healthcare professional to determine the appropriate dosage and
treatment plan for a teenager with bipolar disorder. They will consider factors such as the child's
age, weight, and medical history to ensure safe and effective treatment.

Additionally, it's crucial to monitor the child's response to the medication and adjust the dosage
as needed. Lithium Carbonate can have side effects, and regular check-ups with a healthcare
professional are necessary to minimize potential risks.

Remember, always consult a healthcare professional before starting any medication, especially for
children and teenagers. They will provide personaliz

In [30]:
!pip install fastapi nest-asyncio pyngrok uvicorn


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.2/92.2 kB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.7/307.7 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.4/341.4 kB 19.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 59.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 14.8 MB/s eta 0:00:00


In [31]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip

--2024-07-19 04:30:42--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 54.161.241.46, 52.202.168.65, 18.205.222.128, ...
Connecting to bin.equinox.io (bin.equinox.io)|54.161.241.46|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13921656 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  13.28M  34.8MB/s    in 0.4s    

2024-07-19 04:30:42 (34.8 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13921656/13921656]



In [32]:
!unzip -o ngrok-stable-linux-amd64.zip

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


In [33]:
from pyngrok import conf
os.environ["NGROK"] = userdata.get("NGROK")
conf.get_default().auth_token = os.environ["NGROK"]

In [34]:
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
from fastapi.middleware.cors import CORSMiddleware

app = FastAPI()

class QuestionRequest(BaseModel):
    question: str

app.add_middleware(
    CORSMiddleware,
    allow_origins=['*'],
    allow_credentials=True,
    allow_methods=['*'],
    allow_headers=['*'],
)


@app.post("/ask")
async def ask_question(request: QuestionRequest):
    try:
        response = qa.invoke(request.question)
        response_txt = response["result"]
        return {"answer": response_txt}
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

@app.get('/')
async def root():
    return {'hello': 'world'}

In [ ]:
import nest_asyncio
from pyngrok import ngrok
import uvicorn

ngrok_tunnel = ngrok.connect(8000)
print('Public URL:', ngrok_tunnel.public_url)
nest_asyncio.apply()
uvicorn.run(app, port=8000)

Public URL: https://5fcc-35-192-181-148.ngrok-free.app


INFO:     Started server process [363]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)
